In [1]:
# Identify athletes meeting OCTC selection rules

#1. At least 1 athlete per event per gender
#2. Except for 100m, 400m, cap at 3 athletes per event. 100m, 400m capped at 6
#3. Where top athlete is >30 yrs old (except marathon), to include next athlete as well (below 30)
#4. Where althlete qualified in 2 events, to choose the better performing one
#5. For athletes looking to do full time, to write in to SAA for special consideration
#6. Exclude SPEX carded athletes
#7. Except for marathon, age threshold cut off of 40 yrs old for top athlete
#8. No double tapping of prog - potential names in red


%load_ext autoreload
%autoreload 2

In [180]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import datetime
import icecream as ic

from google.cloud import storage



In [983]:
# PRODUCTION ENVIRONMENT
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, TEAM, AGE, RANK AS COMPETITION_RANK, DIVISION, EVENT, DISTANCE, EVENT_CLASS, UNIQUE_ID, DOB, NATIONALITY, WIND, CATEGORY_EVENT, GENDER, COMPETITION, YEAR, REGION
FROM `saa-analytics.results.athlete_results_prod` 

"""

competitors = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [984]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,YEAR,REGION
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,24/11/90,nan,,Throw,Male,54th SA Inter Club Championships 2024,2024,Local
1,"Ibrahim, Ahmad Arif",DNS,MOE,31,0,M Elite,Run,2400,,nan,20/9/93,nan,,None,Male,Pocari Sweat Run 2.4km 2024,2024,Local
2,"Ho, Kenneth",DNS,NTU,26,0,Open,Steeplechase,3000,(0.914m),nan,14/4/98,nan,,Steeple,Male,SA Allcomers Meet 1,2024,Local
3,"Ryan, Praharsh",DNS,NTU,26,0,Open,Dash,100,,nan,18/10/98,nan,1.8,Sprint,Male,SA Allcomers Meet 1,2024,Local
4,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,4/5/95,nan,,Jump,Male,54th SA Inter Club Championships 2024,2024,Local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21960,"{'126': 'Wei Liang, Leow', '127': 'Weijie, Lim...",DNF,Singapore University of Social,nan,0,Open,Relay,400,,,nan,nan,,Relay,Male,54th SA Inter Club Championships 2024,2024,Local
21961,"{'126': 'Wei Liang, Leow', '127': 'Weijie, Lim...",DNF,Singapore University of Social,nan,0,Open,Relay,400,,,nan,nan,,Relay,Male,54th SA Inter Club Championships 2024,2024,Local
21962,"{'77': 'Abdul Hakim Bin Abdu, Hakim', '79': 'G...",46.33,Singapore University of Social,nan,8,Open,Relay,400,,,nan,nan,,Relay,Male,IVP Track & Field Championships 2024,2024,Local
21963,"{'169': 'Nakamura, Yuna', '171': 'Matsui, Yuri...",01:02.1,Waseda Shibuya Senior High Sch,nan,7,Open,Relay,400,,,nan,nan,,Relay,Female,11th Club ZOOM Kindred Spirit 2024 & PEERS,2024,Local


In [985]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


competitors.to_csv('athletes_new_format_prod.csv', sep=',', encoding='utf-8-sig', index=False)

In [986]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,YEAR,REGION
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,24/11/90,nan,,Throw,Male,54th SA Inter Club Championships 2024,2024,Local
1,"Ibrahim, Ahmad Arif",DNS,MOE,31,0,M Elite,Run,2400,,nan,20/9/93,nan,,None,Male,Pocari Sweat Run 2.4km 2024,2024,Local
2,"Ho, Kenneth",DNS,NTU,26,0,Open,Steeplechase,3000,(0.914m),nan,14/4/98,nan,,Steeple,Male,SA Allcomers Meet 1,2024,Local
3,"Ryan, Praharsh",DNS,NTU,26,0,Open,Dash,100,,nan,18/10/98,nan,1.8,Sprint,Male,SA Allcomers Meet 1,2024,Local
4,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,4/5/95,nan,,Jump,Male,54th SA Inter Club Championships 2024,2024,Local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21960,"{'126': 'Wei Liang, Leow', '127': 'Weijie, Lim...",DNF,Singapore University of Social,nan,0,Open,Relay,400,,,nan,nan,,Relay,Male,54th SA Inter Club Championships 2024,2024,Local
21961,"{'126': 'Wei Liang, Leow', '127': 'Weijie, Lim...",DNF,Singapore University of Social,nan,0,Open,Relay,400,,,nan,nan,,Relay,Male,54th SA Inter Club Championships 2024,2024,Local
21962,"{'77': 'Abdul Hakim Bin Abdu, Hakim', '79': 'G...",46.33,Singapore University of Social,nan,8,Open,Relay,400,,,nan,nan,,Relay,Male,IVP Track & Field Championships 2024,2024,Local
21963,"{'169': 'Nakamura, Yuna', '171': 'Matsui, Yuri...",01:02.1,Waseda Shibuya Senior High Sch,nan,7,Open,Relay,400,,,nan,nan,,Relay,Female,11th Club ZOOM Kindred Spirit 2024 & PEERS,2024,Local


In [987]:
competitors[competitors['NAME']=='SOH RUI YONG GUILLAUME']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,YEAR,REGION
874,SOH RUI YONG GUILLAUME,04:59.3,INDIVIDUAL,33,12,None,1 mile,None,None,None,1991,SGP,-,Mid,Male,Vitaly West Mininster Mile 5K (UK),2024,International
876,SOH RUI YONG GUILLAUME,31:11.4,INDIVIDUAL,33,133,None,10000m,None,None,None,1991,SGP,-,Long,Male,"Tokai University Long Distance Challenge 10,0...",2024,International
877,SOH RUI YONG GUILLAUME,15:07.7,INDIVIDUAL,33,46,None,5000m,None,None,None,1991,SGP,-,Long,Male,Friday Night Under the Lights 5K (UK),2024,International
878,SOH RUI YONG GUILLAUME,08:35.8,INDIVIDUAL,33,None,None,3000m,None,None,None,1991,SGP,-,Long,Male,Hercules Wimbledon Athletics Club 3000m,2024,International


In [1137]:
athletes_2024 = competitors[competitors['YEAR']=='2024']

In [1138]:
athletes_2024

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,YEAR,REGION
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,24/11/90,nan,,Throw,Male,54th SA Inter Club Championships 2024,2024,Local
1,"Ibrahim, Ahmad Arif",DNS,MOE,31,0,M Elite,Run,2400,,nan,20/9/93,nan,,None,Male,Pocari Sweat Run 2.4km 2024,2024,Local
2,"Ho, Kenneth",DNS,NTU,26,0,Open,Steeplechase,3000,(0.914m),nan,14/4/98,nan,,Steeple,Male,SA Allcomers Meet 1,2024,Local
3,"Ryan, Praharsh",DNS,NTU,26,0,Open,Dash,100,,nan,18/10/98,nan,1.8,Sprint,Male,SA Allcomers Meet 1,2024,Local
4,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,4/5/95,nan,,Jump,Male,54th SA Inter Club Championships 2024,2024,Local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21960,"{'126': 'Wei Liang, Leow', '127': 'Weijie, Lim...",DNF,Singapore University of Social,nan,0,Open,Relay,400,,,nan,nan,,Relay,Male,54th SA Inter Club Championships 2024,2024,Local
21961,"{'126': 'Wei Liang, Leow', '127': 'Weijie, Lim...",DNF,Singapore University of Social,nan,0,Open,Relay,400,,,nan,nan,,Relay,Male,54th SA Inter Club Championships 2024,2024,Local
21962,"{'77': 'Abdul Hakim Bin Abdu, Hakim', '79': 'G...",46.33,Singapore University of Social,nan,8,Open,Relay,400,,,nan,nan,,Relay,Male,IVP Track & Field Championships 2024,2024,Local
21963,"{'169': 'Nakamura, Yuna', '171': 'Matsui, Yuri...",01:02.1,Waseda Shibuya Senior High Sch,nan,7,Open,Relay,400,,,nan,nan,,Relay,Female,11th Club ZOOM Kindred Spirit 2024 & PEERS,2024,Local


In [1139]:
athletes_2024[athletes_2024['NAME']=='SOH RUI YONG GUILLAUME']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,YEAR,REGION
874,SOH RUI YONG GUILLAUME,04:59.3,INDIVIDUAL,33,12,None,1 mile,None,None,None,1991,SGP,-,Mid,Male,Vitaly West Mininster Mile 5K (UK),2024,International
876,SOH RUI YONG GUILLAUME,31:11.4,INDIVIDUAL,33,133,None,10000m,None,None,None,1991,SGP,-,Long,Male,"Tokai University Long Distance Challenge 10,0...",2024,International
877,SOH RUI YONG GUILLAUME,15:07.7,INDIVIDUAL,33,46,None,5000m,None,None,None,1991,SGP,-,Long,Male,Friday Night Under the Lights 5K (UK),2024,International
878,SOH RUI YONG GUILLAUME,08:35.8,INDIVIDUAL,33,None,None,3000m,None,None,None,1991,SGP,-,Long,Male,Hercules Wimbledon Athletics Club 3000m,2024,International


In [1140]:
athletes_2024.to_csv('athletes_2024_prod.csv', encoding='utf-8')

In [1141]:
# Choose 2024 only

athletes = athletes_2024

In [1142]:
athletes

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,YEAR,REGION
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,24/11/90,nan,,Throw,Male,54th SA Inter Club Championships 2024,2024,Local
1,"Ibrahim, Ahmad Arif",DNS,MOE,31,0,M Elite,Run,2400,,nan,20/9/93,nan,,None,Male,Pocari Sweat Run 2.4km 2024,2024,Local
2,"Ho, Kenneth",DNS,NTU,26,0,Open,Steeplechase,3000,(0.914m),nan,14/4/98,nan,,Steeple,Male,SA Allcomers Meet 1,2024,Local
3,"Ryan, Praharsh",DNS,NTU,26,0,Open,Dash,100,,nan,18/10/98,nan,1.8,Sprint,Male,SA Allcomers Meet 1,2024,Local
4,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,4/5/95,nan,,Jump,Male,54th SA Inter Club Championships 2024,2024,Local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21960,"{'126': 'Wei Liang, Leow', '127': 'Weijie, Lim...",DNF,Singapore University of Social,nan,0,Open,Relay,400,,,nan,nan,,Relay,Male,54th SA Inter Club Championships 2024,2024,Local
21961,"{'126': 'Wei Liang, Leow', '127': 'Weijie, Lim...",DNF,Singapore University of Social,nan,0,Open,Relay,400,,,nan,nan,,Relay,Male,54th SA Inter Club Championships 2024,2024,Local
21962,"{'77': 'Abdul Hakim Bin Abdu, Hakim', '79': 'G...",46.33,Singapore University of Social,nan,8,Open,Relay,400,,,nan,nan,,Relay,Male,IVP Track & Field Championships 2024,2024,Local
21963,"{'169': 'Nakamura, Yuna', '171': 'Matsui, Yuri...",01:02.1,Waseda Shibuya Senior High Sch,nan,7,Open,Relay,400,,,nan,nan,,Relay,Female,11th Club ZOOM Kindred Spirit 2024 & PEERS,2024,Local


In [1143]:
athletes[athletes['NAME']=='Jun Jie Calvin Quek']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,YEAR,REGION
1939,Jun Jie Calvin Quek,53.19,None,None,4,None,400m Hurdles,None,None,None,26-Feb-96,SGP,None,Hurdles,Male,UAE Athletics Grand Prix,2024,International
1940,Jun Jie Calvin Quek,52.05,None,None,8,None,400m Hurdles,None,None,None,26-Feb-96,SGP,None,Hurdles,Male,11th Kinami Michitaka Memorial Athletics Meet,2024,International
2042,Jun Jie Calvin Quek,51.8,None,None,1,None,400m Hurdles,None,None,None,26-Feb-96,SGP,None,Hurdles,Male,Taiwan Athletics Open,2024,International
2070,Jun Jie Calvin Quek,51.79,None,None,2,None,400m Hurdles,None,None,None,26-Feb-96,SGP,None,Hurdles,Male,XXXIII Qosanov Memorial,2024,International
2071,Jun Jie Calvin Quek,56.95,None,None,3,None,400m Hurdles,None,None,None,26-Feb-96,SGP,None,Hurdles,Male,XXXIII Qosanov Memorial,2024,International
2217,Jun Jie Calvin Quek,52.18,None,None,3,None,400m Hurdles,None,None,None,26-Feb-96,SGP,None,Hurdles,Male,Second Belt and Road Invitational Athletics Me...,2024,International
2218,Jun Jie Calvin Quek,51.33,None,None,2,None,400m Hurdles,None,None,None,26-Feb-96,SGP,None,Hurdles,Male,Second Belt and Road Invitational Athletics Me...,2024,International


In [1144]:
# Run events

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+200 Meter Dash.+', value='200m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+100 Meter Dash.+', value='100m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400 Meter Dash.+', value='400m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+800 Meter Run.+', value='800m')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+1500 Meter Run.+', value='1500m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 Meter Run.+', value='3000m')

# Hurdles events

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+110 Meter Hurdles.+', value='110m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+110m Hurdles.+', value='110m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+100 Meter Hurdles.+', value='100m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+200 Meter Hurdles.+', value='200m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400 Meter Hurdles.+', value='400m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400m Hurdles.+', value='400m hurdles')


#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+4x100 Meter Relay.+', value='4 x 100m relay')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+4x400 Meter Relay.+', value='4 x 400m relay')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 meter.+', value='3000m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+1500 Meter Race Walk.+', value='1500m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000m Race Walk.+', value='3000m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000 Meter Race Walk.+', value='5000m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000m Race Walk.+', value='5000m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+10000 Meter Race Walk.+', value='10000m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000 Meter Run.+', value='5000m run')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+10000 Meter Run.+', value='10000m run')


#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Race Walk.+', value='race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Pole Vault.+', value='Pole vault')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Shot Put.+', value='Shot put')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Discus.+', value='Discus throw')

#mask = athletes['EVENT'].str.contains(r'Discus', na=True)
#athletes.loc[mask, 'EVENT'] = 'Discus throw'

#mask = athletes['EVENT'].str.contains(r'Shot', na=True)
#athletes.loc[mask, 'EVENT'] = 'Shot put'

#mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
#athletes.loc[mask, 'EVENT'] = 'Javelin throw'



#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Triple Jump.+', value='Triple jump')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Javelin Throw.+', value='Javelin throw')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Long Jump.+', value='Long jump')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'Long Jump', value='Long jump')


#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'High Jump', value='High jump')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump.+', value='High jump')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+S/C.+', value='steeplechase')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 Meter Steeplechase.+', value='3000m steeplechase')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+2000 Meter Steeplechase.+', value='2000m steeplechase')


#mask = athletes['EVENT'].str.contains(r'High', na=True)
#athletes.loc[mask, 'EVENT'] = 'High jump'

#mask = athletes['EVENT'].str.contains(r'110m hurdles', na=True)
#athletes.loc[mask, 'EVENT'] = '110m hurdles'

#mask = athletes['EVENT'].str.contains(r'400m hurdles', na=True)
#athletes.loc[mask, 'EVENT'] = '400m hurdles'

#mask = athletes['EVENT'].str.contains(r'200m Hurdles', na=True)
#athletes.loc[mask, 'EVENT'] = '200m hurdles'

#mask = athletes['EVENT'].str.contains(r'100m Hurdles', na=True)
#athletes.loc[mask, 'EVENT'] = '100m hurdles'

#mask = athletes['EVENT'].str.contains(r'4 X 100m relay', na=True)
#athletes.loc[mask, 'EVENT'] = '4 x 100m relay'

#mask = athletes['EVENT'].str.contains(r'4 X 400m relay', na=True)
#athletes.loc[mask, 'EVENT'] = '4 x 400m relay'

#mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
#athletes.loc[mask, 'EVENT'] = '2000m steeplechase'

#mask = athletes['EVENT'].str.contains(r'Hammer Throw', na=True)
#athletes.loc[mask, 'EVENT'] = 'Hammer throw'

#mask = athletes['EVENT'].str.contains(r'3000m S/C', na=True)
#athletes.loc[mask, 'EVENT'] = '3000m steeplechase'

#mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
#athletes.loc[mask, 'EVENT'] = '2000m steeplechase'


#mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
#athletes.loc[mask, 'EVENT'] = '4 x 100m relay'

#mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
#athletes.loc[mask, 'EVENT'] = '4 x 400m relay'


# correct javelin category
#mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
#athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'




In [1145]:
# Wind mapping

#mask = athletes['EVENT'].str.contains(r'\d{1}.\d{1}', na=True, regex=True)
#athletes.loc[mask, 'WIND_MAP'] = '400m'


In [1146]:
# Remove special characters before mapping

for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()

# Create temporary mapped event column

athletes['MAPPED_EVENT']=''

# Correct javelin category

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'


# Running

#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'50', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '50m'
#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'60', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '60m'
#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'80', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m'
mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'100 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'^100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'^200m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'200\sMeter', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'300 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '300m'
mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'400', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'

mask = athletes['EVENT'].str.contains(r'^400\sMeter$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'^600$', na=True, regex=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '600m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'800', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'800 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'^800m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1000m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1500', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'^1500m$', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'3000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
#mask = athletes['EVENT'].str.contains(r'3000m', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'5000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'5000m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'10000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'
mask = athletes['EVENT'].str.contains(r'^10000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'Mile', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1 Mile'

mask = athletes['EVENT'].str.contains(r'10\,000m', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'



# Hurdles

#mask = athletes['EVENT'].str.contains(r'^80m\sHurdles$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'
#mask = athletes['EVENT'].str.contains(r'^80m\shurdles$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'
#mask = athletes['EVENT'].str.contains(r'^80\sMeter\sHurdles$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'80', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'

#mask = athletes['EVENT'].str.contains(r'100\sMeter\sHurdles\s\(0\.838m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
#mask = athletes['EVENT'].str.contains(r'100m\sHurdles\s\(0\.838m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'


##mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(0\.914m\)', na=True)
##athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
##mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(1\.067m\)', na=True)
##athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('0.84', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['DIVISION'].str.contains('None', na=False) & athletes['GENDER'].str.contains(r'Female', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'100', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'

mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.838|0.84', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = ((athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False)) 
         & ((athletes['EVENT_CLASS'].str.contains('None', na=False))|(athletes['EVENT_CLASS']==np.nan)|(athletes['EVENT_CLASS']=='')) 
         & athletes['REGION'].str.contains(r'International', na=False) & (athletes['DIVISION'].str.contains(r'None', na=False)))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'

# Using np.where instead

#athletes['MAPPED_EVENT'] = np.where(((athletes['EVENT']=='110m hurdles|110m Hurdles') & ((athletes['EVENT_CLASS']=='')|athletes['EVENT_CLASS']=='None') & (athletes['REGION']=='International')), '110m Hurdles', ' ')   
                                


mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'1.067', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(' ', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'







#mask = (athletes['EVENT'].str.contains(r'^400m\sHurdles$', na=False) & athletes['EVENT_CLASS'].str.contains(r'', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0.840m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = ' '
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.84|84cm', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


#mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['DIVISION'].str.contains(r'Open|Invitational', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'

mask = (athletes['EVENT'].str.contains(r'400m Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False)  & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


#mask = athletes['EVENT'].str.contains(r'^400\sMeter\sHurdles\s\(0\.762m\)$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
##mask = athletes['EVENT'].str.contains(r'^400m\sHurdles\s\(0\.762m\)$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False)& athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762m', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('None|0.762|0.914', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


# Throws


#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(800g\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'

mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'600g', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'800g', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'


mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS']=='4kg'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'


#mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s4kg\sOpen', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Men\sShot\sPut\s4kg\sOPEN', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'


#mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s\(4kg\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Shot\sPut\s\(7\.26kg\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Shot\sPut\s7\.26kg\sOpen', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['GENDER']=='Male') & (athletes['EVENT_CLASS'].str.contains(r'7.26', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS'].str.contains(r'4', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'



#mask = (athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(4kg\)', na=True) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
#mask = athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(7\.26kg\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'7.26kg', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'4.00kg', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'



#mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=False)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

#mask = ((athletes['EVENT'].str.contains(r'Discus\s\(1\.00kg\)', na=False))  & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'


#mask = athletes['EVENT'].str.contains(r'Discus\sThrow\s\(2kg\)', na=False)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
#mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=False)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'2kg|2.00kg', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'1kg|1.00kg', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

#mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'None', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'



# Jumps

mask = athletes['EVENT'].str.contains(r'High Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'

mask = athletes['EVENT'].str.contains(r'^Long\sJump$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Open', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Trial', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'


mask = athletes['EVENT'].str.contains(r'Triple Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'Pole Vault', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'
mask = athletes['EVENT'].str.contains(r'High jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'
mask = athletes['EVENT'].str.contains(r'Long jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Triple jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'^Pole\svault$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'

# Steeplechase

mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = athletes['EVENT'].str.contains(r'2000m steeplechase', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False)  & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False)  & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'


# Marathon

mask = athletes['EVENT'].str.contains(r'^Marathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\sMarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\smarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'


# Walk

#mask = athletes['EVENT'].str.contains(r'1500m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = athletes['EVENT'].str.contains(r'1500 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'1500', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'


#mask = athletes['EVENT'].str.contains(r'3000m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = athletes['EVENT'].str.contains(r'3000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'


#mask = athletes['EVENT'].str.contains(r'5000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Race Walk'
#mask = athletes['EVENT'].str.contains(r'5000m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'5000', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'


#mask = athletes['EVENT'].str.contains(r'10000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Race Walk'

# Relay

mask = athletes['EVENT'].str.contains(r'4x80m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 80m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4 X 100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'

mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4 X 400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4x100 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'1600', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'

# Decathlon/Heptathlon

mask = athletes['EVENT'].str.contains(r'^Heptathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'^Decathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'
mask = athletes['EVENT'].str.contains(r'Heptathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'Decathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_32475/2824772889.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].astype(str)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_32475/2824772889.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_32475/2824772889.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_32475/2824772889.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_32475/2824772889.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_32475/2824772889.py:8: SettingWithCopyWarning: 
A 

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_32475/2824772889.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_32475/2824772889.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_32475/2824772889.py:8: SettingWithCopyWarning: 
A 

In [1147]:
athletes[(athletes['EVENT']=='110m Hurdles') & (athletes['EVENT_CLASS']=='None')]

#athletes[(athletes['EVENT']=='110m Hurdles')].tail(50)

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,YEAR,REGION,MAPPED_EVENT
1932,Chen Xiang Ang,14.32,None,None,5,None,110m Hurdles,None,None,None,3-Jul-94,SGP,2,Hurdles,Male,International Track Meet,2024,International,110m Hurdles
1933,Chen Xiang Ang,14.14,None,None,6,None,110m Hurdles,None,None,None,3-Jul-94,SGP,0.2,Hurdles,Male,Liese Prokop Memorial,2024,International,110m Hurdles
1934,Chen Xiang Ang,14.09w,None,None,1,None,110m Hurdles,None,None,None,3-Jul-94,SGP,5.1,Hurdles,Male,Lundakvällen,2024,International,110m Hurdles
1935,Chen Xiang Ang,14.37,None,None,8,None,110m Hurdles,None,None,None,3-Jul-94,SGP,0.8,Hurdles,Male,Pärnu Beach Stadium Meeting,2024,International,110m Hurdles
1936,Chen Xiang Ang,14.25w,None,None,6,None,110m Hurdles,None,None,None,3-Jul-94,SGP,2.1,Hurdles,Male,Raiffeisen AUSTRIAN OPEN Eisenstadt,2024,International,110m Hurdles
1937,Chen Xiang Ang,14.26,None,None,2,None,110m Hurdles,None,None,None,3-Jul-94,SGP,1,Hurdles,Male,Venizelia - Chania International Meeting,2024,International,110m Hurdles
1991,Chen Xiang Ang,14.2,None,None,3,None,110m Hurdles,None,None,None,3-Jul-94,SGP,-2.1,Hurdles,Male,AtleticaGeneve,2024,International,110m Hurdles
2009,Chen Xiang Ang,14.3,None,None,3,None,110m Hurdles,None,None,None,3-Jul-94,SGP,-0.9,Hurdles,Male,Ter Specke Bokaal,2024,International,110m Hurdles
2010,Chen Xiang Ang,14.64,None,None,3,None,110m Hurdles,None,None,None,3-Jul-94,SGP,-0.9,Hurdles,Male,Ter Specke Bokaal,2024,International,110m Hurdles
2031,Chen Xiang Ang,14.3,None,None,4,None,110m Hurdles,None,None,None,3-Jul-94,SGP,0.5,Hurdles,Male,Harry Schulting Games,2024,International,110m Hurdles


In [1148]:
athletes[athletes['MAPPED_EVENT']=='400m Hurdles']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,YEAR,REGION,MAPPED_EVENT
109,"Kumaraswamy, K.Balu",DNS,INDIA,50,0,Masters,Hurdles,400,(0.914m),,nan,,,Hurdles,Male,SMTFA Track & Field Championship 2024,2024,Local,400m Hurdles
153,"Vaghela, Rohit Kumar Mangal S",DNS,INDIA,40,0,Masters,Hurdles,400,(0.914m),nan,9/7/83,nan,,Hurdles,Male,SMTFA Track & Field Championship 2024,2024,Local,400m Hurdles
317,"Ng, Lucas Wei Sheng",DNS,SAFSA,34,0,Masters,Hurdles,400,(0.914m),nan,18/7/89,nan,,Hurdles,Male,SMTFA Track & Field Championship 2024,2024,Local,400m Hurdles
379,"., Augustino",01:07.7,SMTFA,45,1,Masters,Hurdles,400,(0.914m),nan,5/8/78,nan,,Hurdles,Male,SMTFA Track & Field Championship 2024,2024,Local,400m Hurdles
398,"Liu, Ada",DNS,SMTFA,50,0,Masters,Hurdles,400,(0.762m),nan,16/4/74,nan,,Hurdles,Female,SMTFA Track & Field Championship 2024,2024,Local,400m Hurdles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21231,"Ng, Silas",01:06.6,Singapore Institute of Managem,24,2,Invitational,Hurdles,400,,nan,4/9/00,nan,,Hurdles,Male,SA Allcomers Meet 3,2024,Local,400m Hurdles
21232,"Ng, Silas",01:07.0,Singapore Institute of Managem,24,3,Open,Hurdles,400,(0.914m),nan,4/9/00,nan,,Hurdles,Male,SA Allcomers Meet 2,2024,Local,400m Hurdles
21376,"BIN MD, MOHAMMAD IRFAN QABEE",DNS,Cougars Athletic Association,25,0,Open,Hurdles,400,,nan,7/2/99,nan,,Hurdles,Male,Pesta Sukan Athletics 2024,2024,Local,400m Hurdles
21390,"Clayton, Caleb",59.43,National University Singapore,25,1,Open,Hurdles,400,(0.914m),nan,1/10/99,nan,,Hurdles,Male,SA Allcomers Meet 2,2024,Local,400m Hurdles


In [1149]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


athletes.to_csv('athletes_post_map_prod.csv', sep=',', encoding='utf-8-sig', index=False)


In [200]:
#credentials = service_account.Credentials.from_service_account_file(
#    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
#)

#sql="""
#SELECT NAME, RESULT, RANK, EVENT, CATEGORY_EVENT, GENDER, COMPETITION, STAGE
#FROM `saa-analytics.results.saa_full`
#WHERE STAGE='Final' AND COMPETITION='SEA Game AND RANK='3'
#"""

#benchmarks = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)




In [202]:
import pandas_gbq
from google.oauth2 import service_account


credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
)

sql="""
SELECT NAME, RESULT, RANK, EVENT, CATEGORY_EVENT, GENDER, COMPETITION, STAGE
FROM `saa-analytics.benchmarks.saa_benchmarks_prod`
WHERE YEAR='2023' AND COMPETITION='Southeast Asian Games' AND RANK='3'
"""

SEAG = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)



Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [203]:
SEAG

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE
0,"{0: '\xa0', 1: ' Jonathan Nyepa, Khairul Hafiz...",39.36,3,4 x 100m,Relay,Male,Southeast Asian Games,None
1,"{0: '\xa0', 1: ' Muhammad Firdaus Bin Mohamad ...",03:08.8,3,4 x 400m,Relay,Male,Southeast Asian Games,None
2,"{0: '\xa0', 1: ' Azreen Nabila Alias, Nur Afri...",44.58,3,4 x 100m,Relay,Female,Southeast Asian Games,None
3,"{0: '\xa0', 1: ' Sukanya Janchaona, Benny Nont...",03:39.3,3,4 x 400m,Relay,Female,Southeast Asian Games,None
4,Mark Lee,21.44,3,200m,Sprint,Male,Southeast Asian Games,Heats
...,...,...,...,...,...,...,...,...
71,Janry Ubas,32.35,3,Discus Throw,Throw,Male,Southeast Asian Games,None
72,Janry Ubas,48.56,3,Javelin Throw,Throw,Male,Southeast Asian Games,None
73,Sarah Dequinan,26.15,3,200m,Sprint,Female,Southeast Asian Games,None
74,Sarah Dequinan,02:29.9,3,800m,Mid,Female,Southeast Asian Games,None


In [204]:
benchmarks=SEAG

In [205]:
benchmarks.rename(columns = {'RESULT':'BENCHMARK'}, inplace = True)


In [206]:
benchmarks.drop(['NAME', 'RANK', 'CATEGORY_EVENT', 'COMPETITION', 'STAGE'], axis=1, inplace=True)


In [207]:
benchmarks

,BENCHMARK,EVENT,GENDER
0,39.36,4 x 100m,Male
1,03:08.8,4 x 400m,Male
2,44.58,4 x 100m,Female
3,03:39.3,4 x 400m,Female
4,21.44,200m,Male
...,...,...,...
71,32.35,Discus Throw,Male
72,48.56,Javelin Throw,Male
73,26.15,200m,Female
74,02:29.9,800m,Female


In [208]:
# Converts any time format into seconds

def convert_time(i, string, metric):

    global output
    
    l=['discus', 'throw', 'jump', 'vault', 'shot']
        
    string=string.lower()
    
    print('metric', metric)
    
    try:
        
        if 'w' in metric:  # skip marks with illegal wind speeds
            
            print('W', metric)
            
            output=''
            
        else:
            
    
            if any(s in string for s in l)==True:
            
                if 'm' in metric:
            
                    metric=metric.replace('m', '')
                    output=float(str(metric))
            
                elif 'GR' in metric:
            
                    metric=metric.replace('GR', '')
                    output=float(str(metric))
                
          #  elif 'w' in metric:
                
               
           #     metric1=metric.replace('w', '')
                
           #     output=float(str(metric1))
                
                else:
    
                    output=float(str(metric))
        
        #elif 'w' in metric:
            
                
         #       metric1=metric.replace('w', '')
                             
         #       output=float(str(metric1))
                
                
        
            else:
        
                searchstring = ":"
                searchstring2 = "."
                substring=str(metric)
                count = substring.count(searchstring)
                count2 = substring.count(searchstring2)
            
                if count==0:
                
                    output=float(substring)
            
            
      #      elif 'marathon' in string and count==2:
                
      #          print('HERE', metric)
                
      #          metric = metric.replace(":", ".", 2)
                
      #          print('METRIC', metric)
                
       #         h,m,s = metric.split(':')            

       #         output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())

               
             
                elif (type(metric)==datetime.time or type(metric)==datetime.datetime):
                
                                                
                    time=str(metric)
                    h, m ,s = time.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                                
                elif (count==1 and count2==1):
            
                    m,s = metric.split(':')
                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                     
                elif (count==1 and count2==2):
                
            
                    metric = metric.replace(".", ":", 1)
            
                    h,m,s = metric.split(':')            
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
                
        
                elif (count==2 and count2==0):
                
            
                    h,m,s = metric.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
  
            

    except:
        
        pass
                
    return output

In [209]:
for i in range(len(benchmarks)):
        
    rowIndex = benchmarks.index[i]

    input_string=benchmarks.iloc[rowIndex,1]
    
    metric=benchmarks.iloc[rowIndex,0]
    
    if metric==None:
        continue
        
    out = convert_time(i, input_string, metric)
     
    benchmarks.loc[rowIndex, 'Metric'] = out

metric 39.36
metric 03:08.8
metric 44.58
metric 03:39.3
metric 21.44
metric 46.63
metric 31:25.5
metric 08:55.0
metric 13.85
metric 50.75
metric 2.15
metric 17.3
metric 24.2
metric 24.13
metric 53.84
metric 04:26.3
metric 35:31.0
metric 59.09
metric 13.46
metric 15.46
metric 21.02
metric 21.26
metric 47.76
metric 01:53.9
metric 01:54.9
metric 01:53.3
metric 14:43.4
metric 50.02
metric 66.2
metric 6891
metric 11.69
metric 02:09.2
metric 17:13.6
metric 11:00.9
metric 13.92
metric 1.73
metric 45.08
metric 5253
metric 11.16
metric 1.84
metric 4.2
metric 7.13
metric 14.76
metric 1.69
metric 10.59
metric 40.59
metric 10.45
metric 10.45
metric 10.66
metric 10.52
metric 47.17
metric 03:59.4
metric 54.76
metric 53.71
metric 5.2
metric 7.62
metric 15.7
metric 59.76
metric 11.75
metric 12.04
metric 23.6
metric 13.59
metric 14.23
metric 4
metric 6.02
metric 14.44
metric 49.61
metric 48.31
metric 54.08
metric 05:51.8
metric 10.95
metric 32.35
metric 48.56
metric 26.15
metric 02:29.9
metric 5.74


In [210]:
benchmarks

,BENCHMARK,EVENT,GENDER,Metric
0,39.36,4 x 100m,Male,39.36
1,03:08.8,4 x 400m,Male,188.80
2,44.58,4 x 100m,Female,44.58
3,03:39.3,4 x 400m,Female,219.30
4,21.44,200m,Male,21.44
...,...,...,...,...
71,32.35,Discus Throw,Male,32.35
72,48.56,Javelin Throw,Male,48.56
73,26.15,200m,Female,26.15
74,02:29.9,800m,Female,149.90


In [211]:
mask = benchmarks['EVENT'].str.contains(r'jump|throw|Pole|put|Jump|Throw|pole|Put', na=True)

benchmarks.loc[mask, '2%']=benchmarks['Metric']*0.98
benchmarks.loc[mask, '3.5%']=benchmarks['Metric']*0.965
benchmarks.loc[mask, '5%']=benchmarks['Metric']*0.95

benchmarks.loc[~mask, '2%']=benchmarks['Metric']*1.02
benchmarks.loc[~mask, '3.5%']=benchmarks['Metric']*1.035
benchmarks.loc[~mask, '5%']=benchmarks['Metric']*1.05


#benchmarks.iloc[5, [1]]='10000m run'
#benchmarks.iloc[28, [1]]='10000m run'
#benchmarks.iloc[26, [1]]='1500m'


In [212]:
benchmarks['MAPPED_EVENT']=benchmarks['EVENT'].str.strip()

In [213]:
benchmarks

,BENCHMARK,EVENT,GENDER,Metric,2%,3.5%,5%,MAPPED_EVENT
0,39.36,4 x 100m,Male,39.36,40.1472,40.73760,41.3280,4 x 100m
1,03:08.8,4 x 400m,Male,188.80,192.5760,195.40800,198.2400,4 x 400m
2,44.58,4 x 100m,Female,44.58,45.4716,46.14030,46.8090,4 x 100m
3,03:39.3,4 x 400m,Female,219.30,223.6860,226.97550,230.2650,4 x 400m
4,21.44,200m,Male,21.44,21.8688,22.19040,22.5120,200m
...,...,...,...,...,...,...,...,...
71,32.35,Discus Throw,Male,32.35,31.7030,31.21775,30.7325,Discus Throw
72,48.56,Javelin Throw,Male,48.56,47.5888,46.86040,46.1320,Javelin Throw
73,26.15,200m,Female,26.15,26.6730,27.06525,27.4575,200m
74,02:29.9,800m,Female,149.90,152.8980,155.14650,157.3950,800m


In [214]:
athletes

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,YEAR,REGION,MAPPED_EVENT
0,"FUN, LE CONG LUCAS",15.35,Singapore,18,1,Hurdles,,1/11/06,,-0.4,Hurdles,Male,SA Trial 19 Jun 2024,2024,Local,
1,"TAN, DARYL HONG AN",7.16,Singapore,23,1,Dash,,18/12/01,,-0.5,Sprint,Male,SA Trial 19 Jun 2024,2024,Local,
2,"HENG, FU HONG BRANDON",NH,Singapore,26,0,High Jump,,13/12/98,,,Jump,Male,SA Trial 19 Jun 2024,2024,Local,High jump
3,"Toh, Tedd",6.55m,Singapore,0,1,Long Jump,,nan,,-0.3,Jump,Male,SA Trial 19 Jun 2024,2024,Local,Long jump
4,"CHEW, YIU TSE JADE",5.02m,Singapore,20,1,Long Jump,,14/1/04,,-0.3,Jump,Female,SA Trial 19 Jun 2024,2024,Local,Long jump
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21960,"Yeo, Xin Ying Natalie Gra",05:47.1,National Junior College,13,1,Run,nan,18/8/11,nan,,Mid,Female,SA Allcomers Meet 1,2024,Local,
21961,"., Zachery Caleb Wong K",59.15,Unity Secondary,15,38,Dash,nan,1/1/09,nan,,Sprint,Male,11th Club ZOOM Kindred Spirit 2024 & PEERS,2024,Local,
21962,"tang, zheng jia ning eliza",01:06.7,Team Fabian,17,4,Dash,nan,9/3/07,nan,,Sprint,Female,Pesta Sukan Athletics 2024,2024,Local,
21963,"tang, zheng jia ning eliza",32.42,Team Fabian,17,10,Dash,nan,9/3/07,nan,0.1,Sprint,Female,Pesta Sukan Athletics 2024,2024,Local,


In [215]:
athletes[athletes['NAME']=='Jun Jie Calvin Quek']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,YEAR,REGION,MAPPED_EVENT
4206,Jun Jie Calvin Quek,53.19,None,None,4,400m Hurdles,None,26-Feb-96,SGP,None,Hurdles,Male,UAE Athletics Grand Prix,2024,International,400m hurdles
4207,Jun Jie Calvin Quek,52.05,None,None,8,400m Hurdles,None,26-Feb-96,SGP,None,Hurdles,Male,11th Kinami Michitaka Memorial Athletics Meet,2024,International,400m hurdles
4542,Jun Jie Calvin Quek,51.8,None,None,1,400m Hurdles,None,26-Feb-96,SGP,None,Hurdles,Male,Taiwan Athletics Open,2024,International,400m hurdles
4570,Jun Jie Calvin Quek,51.79,None,None,2,400m Hurdles,None,26-Feb-96,SGP,None,Hurdles,Male,XXXIII Qosanov Memorial,2024,International,400m hurdles
4571,Jun Jie Calvin Quek,56.95,None,None,3,400m Hurdles,None,26-Feb-96,SGP,None,Hurdles,Male,XXXIII Qosanov Memorial,2024,International,400m hurdles
4713,Jun Jie Calvin Quek,52.18,None,None,3,400m Hurdles,None,26-Feb-96,SGP,None,Hurdles,Male,Second Belt and Road Invitational Athletics Me...,2024,International,400m hurdles
4714,Jun Jie Calvin Quek,51.33,None,None,2,400m Hurdles,None,26-Feb-96,SGP,None,Hurdles,Male,Second Belt and Road Invitational Athletics Me...,2024,International,400m hurdles


In [216]:
# There is a problem with RESULTS column being changed after this statement

#df = athletes.reset_index().merge(benchmarks.reset_index(), on=['MAPPED_EVENT','GENDER'], how='left')
#df = athletes.merge(benchmarks, on=['EVENT','GENDER'], how='left')


In [217]:
# Merge benchmarks onto athletes on MAPPED_EVENT and GENDER

df = pd.merge(
    left=athletes, 
    right=benchmarks,
    how='left',
    left_on=['MAPPED_EVENT', 'GENDER'],
    right_on=['MAPPED_EVENT', 'GENDER'],
)

In [218]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,NATIONALITY,WIND,...,COMPETITION,YEAR,REGION,MAPPED_EVENT,BENCHMARK,EVENT_y,Metric,2%,3.5%,5%
0,"FUN, LE CONG LUCAS",15.35,Singapore,18,1,Hurdles,,1/11/06,,-0.4,...,SA Trial 19 Jun 2024,2024,Local,,NaN,NaN,NaN,NaN,NaN,NaN
1,"TAN, DARYL HONG AN",7.16,Singapore,23,1,Dash,,18/12/01,,-0.5,...,SA Trial 19 Jun 2024,2024,Local,,NaN,NaN,NaN,NaN,NaN,NaN
2,"HENG, FU HONG BRANDON",NH,Singapore,26,0,High Jump,,13/12/98,,,...,SA Trial 19 Jun 2024,2024,Local,High jump,NaN,NaN,NaN,NaN,NaN,NaN
3,"Toh, Tedd",6.55m,Singapore,0,1,Long Jump,,nan,,-0.3,...,SA Trial 19 Jun 2024,2024,Local,Long jump,NaN,NaN,NaN,NaN,NaN,NaN
4,"CHEW, YIU TSE JADE",5.02m,Singapore,20,1,Long Jump,,14/1/04,,-0.3,...,SA Trial 19 Jun 2024,2024,Local,Long jump,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28047,"Yeo, Xin Ying Natalie Gra",05:47.1,National Junior College,13,1,Run,nan,18/8/11,nan,,...,SA Allcomers Meet 1,2024,Local,,NaN,NaN,NaN,NaN,NaN,NaN
28048,"., Zachery Caleb Wong K",59.15,Unity Secondary,15,38,Dash,nan,1/1/09,nan,,...,11th Club ZOOM Kindred Spirit 2024 & PEERS,2024,Local,,NaN,NaN,NaN,NaN,NaN,NaN
28049,"tang, zheng jia ning eliza",01:06.7,Team Fabian,17,4,Dash,nan,9/3/07,nan,,...,Pesta Sukan Athletics 2024,2024,Local,,NaN,NaN,NaN,NaN,NaN,NaN
28050,"tang, zheng jia ning eliza",32.42,Team Fabian,17,10,Dash,nan,9/3/07,nan,0.1,...,Pesta Sukan Athletics 2024,2024,Local,,NaN,NaN,NaN,NaN,NaN,NaN


In [219]:
df[df['NAME']=='Jun Jie Calvin Quek']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,NATIONALITY,WIND,...,COMPETITION,YEAR,REGION,MAPPED_EVENT,BENCHMARK,EVENT_y,Metric,2%,3.5%,5%
4225,Jun Jie Calvin Quek,53.19,None,None,4,400m Hurdles,None,26-Feb-96,SGP,None,...,UAE Athletics Grand Prix,2024,International,400m hurdles,NaN,NaN,NaN,NaN,NaN,NaN
4226,Jun Jie Calvin Quek,52.05,None,None,8,400m Hurdles,None,26-Feb-96,SGP,None,...,11th Kinami Michitaka Memorial Athletics Meet,2024,International,400m hurdles,NaN,NaN,NaN,NaN,NaN,NaN
4918,Jun Jie Calvin Quek,51.8,None,None,1,400m Hurdles,None,26-Feb-96,SGP,None,...,Taiwan Athletics Open,2024,International,400m hurdles,NaN,NaN,NaN,NaN,NaN,NaN
4967,Jun Jie Calvin Quek,51.79,None,None,2,400m Hurdles,None,26-Feb-96,SGP,None,...,XXXIII Qosanov Memorial,2024,International,400m hurdles,NaN,NaN,NaN,NaN,NaN,NaN
4968,Jun Jie Calvin Quek,56.95,None,None,3,400m Hurdles,None,26-Feb-96,SGP,None,...,XXXIII Qosanov Memorial,2024,International,400m hurdles,NaN,NaN,NaN,NaN,NaN,NaN
5278,Jun Jie Calvin Quek,52.18,None,None,3,400m Hurdles,None,26-Feb-96,SGP,None,...,Second Belt and Road Invitational Athletics Me...,2024,International,400m hurdles,NaN,NaN,NaN,NaN,NaN,NaN
5279,Jun Jie Calvin Quek,51.33,None,None,2,400m Hurdles,None,26-Feb-96,SGP,None,...,Second Belt and Road Invitational Athletics Me...,2024,International,400m hurdles,NaN,NaN,NaN,NaN,NaN,NaN


In [220]:
df[df['MAPPED_EVENT']=='Discus throw']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,NATIONALITY,WIND,...,COMPETITION,YEAR,REGION,MAPPED_EVENT,BENCHMARK,EVENT_y,Metric,2%,3.5%,5%
52,"Lam Chen Hee, Aidan",11.48m,Pathlight School,12,52,Discus Throw,,2/3/12,,,...,nan,2024,Local,Discus throw,NaN,NaN,NaN,NaN,NaN,NaN
64,"Shrivastava, Aditya",10.06m,Tanjong Katong Primary School,12,61,Discus Throw,,20/3/12,,,...,nan,2024,Local,Discus throw,NaN,NaN,NaN,NaN,NaN,NaN
68,"Kwa, Edwin",19.10m,Kheng Cheng School,12,5,Discus Throw,,10/4/12,,,...,nan,2024,Local,Discus throw,NaN,NaN,NaN,NaN,NaN,NaN
72,"Ng Keng Yap, Isaac",12.86m,Club ZOOM,12,40,Discus Throw,,7/12/12,,,...,nan,2024,Local,Discus throw,NaN,NaN,NaN,NaN,NaN,NaN
89,"Jayasekara, Ruwan",DNS,Sri Lanka,50,0,Discus Throw,,12/5/74,,,...,SMTFA Track & Field Championship 2024,2024,Local,Discus throw,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27920,"ABDILLAH, MOHAMED AN'NIQUE ABD",FOUL,Victoria School,15,0,Discus Throw,nan,8/1/09,nan,,...,SA Allcomers Meet 2,2024,Local,Discus throw,NaN,NaN,NaN,NaN,NaN,NaN
27929,"ALLAPITCHAY, MUHAMMAD ALI ALLAPIT",24.45m,Victoria School,14,10,Discus Throw,nan,29/5/10,nan,,...,SA Allcomers Meet 2,2024,Local,Discus throw,NaN,NaN,NaN,NaN,NaN,NaN
27932,"MOHAMAD SHAIFUL AZHA, MUHAMMAD IZDIHAR BIN",25.19m,Victoria School,15,15,Discus Throw,nan,19/2/09,nan,,...,SA Allcomers Meet 2,2024,Local,Discus throw,NaN,NaN,NaN,NaN,NaN,NaN
27937,"PARAMESWARI, MURUGANANTHAM KALIGA",15.60m,Unattached,30,8,Discus Throw,nan,8/8/94,nan,,...,Pesta Sukan Athletics 2024,2024,Local,Discus throw,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
# replace '-' with NaN

df['RESULT'] = df['RESULT'].replace(regex=r'–', value=np.NaN)
#df['SEED'] = df['SEED'].replace(regex=r'–', value=np.NaN)


In [138]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,NATIONALITY,WIND,...,REGION,MAPPED_EVENT,BENCHMARK,EVENT_y,Metric,2%,3.5%,5%,RESULT_CONV,SEED_CONV
0,"FUN, LE CONG LUCAS",15.35,Singapore,18,1,Hurdles,,1/11/06,,-0.4,...,Local,,NaN,NaN,NaN,NaN,NaN,NaN,5.74,15.35
1,"TAN, DARYL HONG AN",7.16,Singapore,23,1,Dash,,18/12/01,,-0.5,...,Local,,NaN,NaN,NaN,NaN,NaN,NaN,15.35,7.16
2,"HENG, FU HONG BRANDON",NH,Singapore,26,0,High Jump,,13/12/98,,,...,Local,High jump,NaN,NaN,NaN,NaN,NaN,NaN,7.16,7.16
3,"Toh, Tedd",6.55m,Singapore,0,1,Long Jump,,nan,,-0.3,...,Local,Long jump,NaN,NaN,NaN,NaN,NaN,NaN,7.16,7.16
4,"CHEW, YIU TSE JADE",5.02m,Singapore,20,1,Long Jump,,14/1/04,,-0.3,...,Local,Long jump,NaN,NaN,NaN,NaN,NaN,NaN,7.16,7.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28047,"Yeo, Xin Ying Natalie Gra",05:47.1,National Junior College,13,1,Run,nan,18/8/11,nan,,...,Local,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28048,"., Zachery Caleb Wong K",59.15,Unity Secondary,15,38,Dash,nan,1/1/09,nan,,...,Local,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28049,"tang, zheng jia ning eliza",01:06.7,Team Fabian,17,4,Dash,nan,9/3/07,nan,,...,Local,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28050,"tang, zheng jia ning eliza",32.42,Team Fabian,17,10,Dash,nan,9/3/07,nan,0.1,...,Local,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
df[df['NAME']=='Jun Jie Calvin Quek']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,NATIONALITY,WIND,...,REGION,MAPPED_EVENT,BENCHMARK,EVENT_y,Metric,2%,3.5%,5%,RESULT_CONV,SEED_CONV
4225,Jun Jie Calvin Quek,53.19,None,None,4,400m Hurdles,None,26-Feb-96,SGP,None,...,International,400m hurdles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4226,Jun Jie Calvin Quek,52.05,None,None,8,400m Hurdles,None,26-Feb-96,SGP,None,...,International,400m hurdles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4918,Jun Jie Calvin Quek,51.8,None,None,1,400m Hurdles,None,26-Feb-96,SGP,None,...,International,400m hurdles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4967,Jun Jie Calvin Quek,51.79,None,None,2,400m Hurdles,None,26-Feb-96,SGP,None,...,International,400m hurdles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4968,Jun Jie Calvin Quek,56.95,None,None,3,400m Hurdles,None,26-Feb-96,SGP,None,...,International,400m hurdles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5278,Jun Jie Calvin Quek,52.18,None,None,3,400m Hurdles,None,26-Feb-96,SGP,None,...,International,400m hurdles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5279,Jun Jie Calvin Quek,51.33,None,None,2,400m Hurdles,None,26-Feb-96,SGP,None,...,International,400m hurdles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:
df.to_csv('df_prod.csv', sep=',', encoding='utf-8-sig', index=False)


In [141]:
# Convert results and seed into seconds format

for i in range(len(df)):
    
    result_out=''
    
        
    rowIndex = df.index[i]

    input_string=df.iloc[rowIndex,6]    # event description
    
    metric=df.iloc[rowIndex,2] # result
    metric_seed=df.iloc[rowIndex, 1]  #seed
    
    if metric=='—' or metric=='DQ' or metric=='SCR' or metric=='FS' or metric=='DNQ' or metric==' DNS' or metric=='NH':
        continue
    
    result_out = convert_time(i, input_string, metric)
    print('line', i, input_string, metric, result_out)

    
#    seed_out = convert_time(i, input_string, metric_seed)
         
    df.loc[rowIndex, 'RESULT_CONV'] = result_out
#    df.loc[rowIndex, 'SEED_CONV'] = seed_out


metric Singapore
line 0   Singapore 319.3
metric 15.35
metric Singapore
line 1   Singapore 15.35
metric 7.16
metric Singapore
line 2   Singapore 7.16
metric NH
metric Singapore
line 3   Singapore 7.16
metric 6.55m
metric Singapore
line 4   Singapore 7.16
metric 5.02m
metric Yangzheng Primary School
line 5   Yangzheng Primary School 7.16
metric 03:33.1
metric ST STEPHEN'S SCHOOL
line 6   ST STEPHEN'S SCHOOL 213.1
metric 7.97m
metric ST STEPHEN'S SCHOOL
line 7   ST STEPHEN'S SCHOOL 213.1
metric 7.43m
metric Nanyang Primary School
line 8   Nanyang Primary School 213.1
metric 1.00m
metric Club ZOOM
line 9   Club ZOOM 213.1
metric 11.49m
metric ActiveSG Club
line 10   ActiveSG Club 213.1
metric 12.99
metric TEAM SAGA
line 11   TEAM SAGA 12.99
metric DNS
metric Club ZOOM
line 12   Club ZOOM 12.99
metric 13.15
metric ActiveSG Club
line 13   ActiveSG Club 13.15
metric 3.30m
metric ActiveSG Club
line 14   ActiveSG Club 13.15
metric 26.09
metric Nanyang Primary School
line 15   Nanyang Primary S

metric SJKC 
line 996 nan SJKC  
metric 1.27m
metric Sembawang Primary School
W Sembawang Primary School
line 997 nan Sembawang Primary School 
metric 1.15m
metric Sembawang Primary School
W Sembawang Primary School
line 998 nan Sembawang Primary School 
metric 1.20m
metric Sembawang Primary School
W Sembawang Primary School
line 999 nan Sembawang Primary School 
metric 1.40m
metric Team Cicada Trackers
line 1000 nan Team Cicada Trackers 
metric 1.20m
metric ActiveSG Club
line 1001 nan ActiveSG Club 
metric 1.30m
metric Sembawang Primary School
W Sembawang Primary School
line 1002 nan Sembawang Primary School 
metric 13.64m
metric Sembawang Primary School
W Sembawang Primary School
line 1003 nan Sembawang Primary School 
metric 11.63m
metric Sembawang Primary School
W Sembawang Primary School
line 1004 nan Sembawang Primary School 
metric 11.41m
metric Sembawang Primary School
W Sembawang Primary School
line 1005 nan Sembawang Primary School 
metric 11.42m
metric Seng Kang Primary Scho

metric Compassvale Primary School
line 1979 nan Compassvale Primary School 243.1
metric 5.09m
metric Compassvale Primary School
line 1980 nan Compassvale Primary School 243.1
metric 11.67
metric Compassvale Primary School
line 1981 nan Compassvale Primary School 11.67
metric 05:11.9
metric Compassvale Primary School
line 1982 nan Compassvale Primary School 311.9
metric 1.69m
metric Compassvale Primary School
line 1983 nan Compassvale Primary School 311.9
metric 4.40m
metric Compassvale Primary School
line 1984 nan Compassvale Primary School 311.9
metric 11.29
metric Compassvale Primary School
line 1985 nan Compassvale Primary School 11.29
metric 04:14.5
metric Compassvale Primary School
line 1986 nan Compassvale Primary School 254.5
metric 2.54m
metric Compassvale Primary School
line 1987 nan Compassvale Primary School 254.5
metric 2.96m
metric Compassvale Primary School
line 1988 nan Compassvale Primary School 254.5
metric 9.9
metric Compassvale Primary School
line 1989 nan Compassval

line 2860 nan INDIA 2251.4
metric DNS
metric INDIA
line 2861 nan INDIA 2251.4
metric DNS
metric INDIA
line 2862 nan INDIA 2251.4
metric 18.88m
metric INDIA
line 2863 nan INDIA 2251.4
metric DNS
metric INDIA
line 2864 nan INDIA 2251.4
metric DNS
metric INDIA
line 2865 nan INDIA 2251.4
metric DNS
metric INDIA
line 2866 nan INDIA 2251.4
metric DNS
metric INDIA
line 2867 nan INDIA 2251.4
metric 24.27m
metric INDIA
line 2868 nan INDIA 2251.4
metric 8.09m
metric INDIA
line 2869 nan INDIA 2251.4
metric 22.96m
metric INDIA
line 2870 nan INDIA 2251.4
metric DNS
metric INDIA
line 2871 nan INDIA 2251.4
metric 6.30m
metric INDIA
line 2872 nan INDIA 2251.4
metric DNS
metric INDIA
line 2873 nan INDIA 2251.4
metric 29.72m
metric INDIA
line 2874 nan INDIA 2251.4
metric 26.64m
metric INDIA
line 2875 nan INDIA 2251.4
metric DNS
metric INDIA
line 2876 nan INDIA 2251.4
metric 37.36m
metric INDIA
line 2877 nan INDIA 2251.4
metric 35.40m
metric INDIA
line 2878 nan INDIA 2251.4
metric 4.45m
metric INDIA
line

metric JS Athletics
line 3840 nan JS Athletics 14.12
metric DNS
metric JS Athletics
line 3841 nan JS Athletics 14.12
metric DNS
metric JS Athletics
line 3842 nan JS Athletics 14.12
metric DNS
metric JS Athletics
line 3843 nan JS Athletics 14.12
metric 11.05
metric JS Athletics
line 3844 nan JS Athletics 11.05
metric 36.59
metric JS Athletics
line 3845 nan JS Athletics 36.59
metric 12.1
metric JS Athletics
line 3846 nan JS Athletics 12.1
metric DNS
metric JS Athletics
line 3847 nan JS Athletics 12.1
metric 11.79
metric JS Athletics
line 3848 nan JS Athletics 11.79
metric 2.00m
metric JS Athletics
line 3849 nan JS Athletics 11.79
metric 41.45
metric JS Athletics
line 3850 nan JS Athletics 41.45
metric DNS
metric JS Athletics
line 3851 nan JS Athletics 41.45
metric DNS
metric JS Athletics
line 3852 nan JS Athletics 41.45
metric 01:27.5
metric JS Athletics
line 3853 nan JS Athletics 87.5
metric 01:39.3
metric JS Athletics
line 3854 nan JS Athletics 99.3
metric 13.73
metric JS Athletics
lin

AttributeError: 'NoneType' object has no attribute 'lower'

In [ ]:
#df["AGE"].fillna(0, inplace=True)
#df['AGE'] = df['AGE'].astype('float')

In [142]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,NATIONALITY,WIND,...,REGION,MAPPED_EVENT,BENCHMARK,EVENT_y,Metric,2%,3.5%,5%,RESULT_CONV,SEED_CONV
0,"FUN, LE CONG LUCAS",15.35,Singapore,18,1,Hurdles,,1/11/06,,-0.4,...,Local,,NaN,NaN,NaN,NaN,NaN,NaN,319.3,15.35
1,"TAN, DARYL HONG AN",7.16,Singapore,23,1,Dash,,18/12/01,,-0.5,...,Local,,NaN,NaN,NaN,NaN,NaN,NaN,15.35,7.16
2,"HENG, FU HONG BRANDON",NH,Singapore,26,0,High Jump,,13/12/98,,,...,Local,High jump,NaN,NaN,NaN,NaN,NaN,NaN,7.16,7.16
3,"Toh, Tedd",6.55m,Singapore,0,1,Long Jump,,nan,,-0.3,...,Local,Long jump,NaN,NaN,NaN,NaN,NaN,NaN,7.16,7.16
4,"CHEW, YIU TSE JADE",5.02m,Singapore,20,1,Long Jump,,14/1/04,,-0.3,...,Local,Long jump,NaN,NaN,NaN,NaN,NaN,NaN,7.16,7.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28047,"Yeo, Xin Ying Natalie Gra",05:47.1,National Junior College,13,1,Run,nan,18/8/11,nan,,...,Local,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28048,"., Zachery Caleb Wong K",59.15,Unity Secondary,15,38,Dash,nan,1/1/09,nan,,...,Local,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28049,"tang, zheng jia ning eliza",01:06.7,Team Fabian,17,4,Dash,nan,9/3/07,nan,,...,Local,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28050,"tang, zheng jia ning eliza",32.42,Team Fabian,17,10,Dash,nan,9/3/07,nan,0.1,...,Local,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [143]:
# Choose SEED if better than RESULT

#condition1=df['SEED_CONV']>df['RESULT_CONV']
#condition2=((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))
#condition3=df['SEED_CONV']<df['RESULT_CONV']
#condition4=~((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))


#df['RESULT_BEST']=df['SEED_CONV'].where((condition1 & condition2)|(condition3 & condition4), df['RESULT_CONV'].values)

df['RESULT_BEST'] = df['RESULT_CONV']

In [144]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,NATIONALITY,WIND,...,MAPPED_EVENT,BENCHMARK,EVENT_y,Metric,2%,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST
0,"FUN, LE CONG LUCAS",15.35,Singapore,18,1,Hurdles,,1/11/06,,-0.4,...,,NaN,NaN,NaN,NaN,NaN,NaN,319.3,15.35,319.3
1,"TAN, DARYL HONG AN",7.16,Singapore,23,1,Dash,,18/12/01,,-0.5,...,,NaN,NaN,NaN,NaN,NaN,NaN,15.35,7.16,15.35
2,"HENG, FU HONG BRANDON",NH,Singapore,26,0,High Jump,,13/12/98,,,...,High jump,NaN,NaN,NaN,NaN,NaN,NaN,7.16,7.16,7.16
3,"Toh, Tedd",6.55m,Singapore,0,1,Long Jump,,nan,,-0.3,...,Long jump,NaN,NaN,NaN,NaN,NaN,NaN,7.16,7.16,7.16
4,"CHEW, YIU TSE JADE",5.02m,Singapore,20,1,Long Jump,,14/1/04,,-0.3,...,Long jump,NaN,NaN,NaN,NaN,NaN,NaN,7.16,7.16,7.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28047,"Yeo, Xin Ying Natalie Gra",05:47.1,National Junior College,13,1,Run,nan,18/8/11,nan,,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28048,"., Zachery Caleb Wong K",59.15,Unity Secondary,15,38,Dash,nan,1/1/09,nan,,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28049,"tang, zheng jia ning eliza",01:06.7,Team Fabian,17,4,Dash,nan,9/3/07,nan,,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28050,"tang, zheng jia ning eliza",32.42,Team Fabian,17,10,Dash,nan,9/3/07,nan,0.1,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
df[df['NAME']=='Jun Jie Calvin Quek']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,NATIONALITY,WIND,...,MAPPED_EVENT,BENCHMARK,EVENT_y,Metric,2%,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST
4225,Jun Jie Calvin Quek,53.19,None,None,4,400m Hurdles,None,26-Feb-96,SGP,None,...,400m hurdles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4226,Jun Jie Calvin Quek,52.05,None,None,8,400m Hurdles,None,26-Feb-96,SGP,None,...,400m hurdles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4918,Jun Jie Calvin Quek,51.8,None,None,1,400m Hurdles,None,26-Feb-96,SGP,None,...,400m hurdles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4967,Jun Jie Calvin Quek,51.79,None,None,2,400m Hurdles,None,26-Feb-96,SGP,None,...,400m hurdles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4968,Jun Jie Calvin Quek,56.95,None,None,3,400m Hurdles,None,26-Feb-96,SGP,None,...,400m hurdles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5278,Jun Jie Calvin Quek,52.18,None,None,3,400m Hurdles,None,26-Feb-96,SGP,None,...,400m hurdles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5279,Jun Jie Calvin Quek,51.33,None,None,2,400m Hurdles,None,26-Feb-96,SGP,None,...,400m hurdles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
# Change to numeric

df[['2%', '3.5%', '5%', 'RESULT_BEST']] = df[['2%', '3.5%', '5%', 'RESULT_BEST']].apply(pd.to_numeric)

In [52]:
mask = df['CATEGORY_EVENT'].str.contains(r'Jump|Throw', na=True)

df.loc[mask, 'Delta2'] = df['RESULT_BEST']-df['2%']
df.loc[mask, 'Delta3.5'] = df['RESULT_BEST']-df['3.5%']
df.loc[mask, 'Delta5'] = df['RESULT_BEST']-df['5%']
df.loc[mask, 'Delta_Benchmark'] = df['RESULT_BEST']-df['Metric']

df.loc[~mask, 'Delta2'] =  df['2%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta3.5'] = df['3.5%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta5'] = df['5%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta_Benchmark'] = df['Metric'] - df['RESULT_BEST']

#rslt_df['Delta2']=rslt_df['2pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta35']=rslt_df['35pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta5']=rslt_df['5pc']-rslt_df['RESULT_CONV']
df=df.loc[df['COMPETITION']!='SEA Games']

In [53]:
# Performance metric to filter out athletes

df['PERF_SCALAR']=df['Delta5']/df['Metric']*100

In [54]:
df.to_csv('df_test_prod.csv', sep=',', encoding='utf-8-sig', index=False)


In [55]:
df[df['NAME']=='LEE, VANESSA']

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,NATIONALITY,...,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
14533,"LEE, VANESSA",05:18.6,04:51.7,SINGAPORE,26,3,Run,nan,23/2/98,nan,...,NaN,NaN,291.7,318.6,291.7,NaN,NaN,NaN,NaN,NaN
14727,"LEE, VANESSA",12:03.4,11:09.2,SINGAPORE,26,1,Steeplechase,nan,23/2/98,nan,...,NaN,NaN,669.2,723.4,669.2,NaN,NaN,NaN,NaN,NaN


In [56]:
'''

# Correct some of the known name variations
# Need to fix 2024 NSG names extracts (extracting Q/q with name)

df['NAME'] = df['NAME'].str.replace('\xa0', '', regex=True)
df['NAME'] = df['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\r', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\n', '', regex=True)
df['NAME'] = df['NAME'].str.strip()



df['NAME'] = df['NAME'].replace(regex=r'Norhisham, Jamie El Redha Ang El Redha Ang', value='Norhisham, Jamie El Redha Ang')



df['NAME'] = df['NAME'].replace(regex=r'TAN, ELIZABETH ANN SHEE R', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Elizabeth Ann', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'TAN, SHEE RU, ELIZABETH-A', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Elizabeth-Ann Tan', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Elizabeth-Ann', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Tan Shee Ru, Elizabeth-Ann', value='TAN, ELIZABETH-ANN')

df['NAME'] = df['NAME'].replace(regex=r'Louis, Marc', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'LOUIS, MARC BRIAN', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'LOUIS, MARC BRIAN BRIAN', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'Louis, Marc Brian Brian', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'Marc Brian Louis', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'Brian louis, Marc', value='Louis, Marc Brian')




df['NAME'] = df['NAME'].replace(regex=r'TAN JUN JIE', value='Tan, Jun Jie')
df['NAME'] = df['NAME'].replace(regex=r'Tan Jun Jie', value='Tan, Jun Jie')

df['NAME'] = df['NAME'].replace(regex=r'Shanti Veronica Pereira', value='PEREIRA, VERONICA SHANTI')
df['NAME'] = df['NAME'].replace(regex=r'LIEW, YEE LING, BERNICE', value='Liew, Bernice')
df['NAME'] = df['NAME'].replace(regex=r'LIEW, BERNICE YEE LING', value='Liew, Bernice')

df['NAME'] = df['NAME'].replace(regex=r'Mark Lee', value='LEE, MARK REN')
df['NAME'] = df['NAME'].replace(regex=r'Lee, Mark', value='LEE, MARK REN')






df['NAME'] = df['NAME'].replace(regex=r'ANG, CHEN XIANG', value='Ang, Chen Xiang')




df['NAME'] = df['NAME'].replace(regex=r'LEE YU FOONG (LI YUFENG)', value='Lee, Yu Foong')
df['NAME'] = df['NAME'].replace(regex=r'YAP, SEAN YEE', value='Yap, Sean Yee')
df['NAME'] = df['NAME'].replace(regex=r'Norhisham, Jamie', value='Norhisham, Jamie El Redha Ang')

df['NAME'] = df['NAME'].replace(regex=r'ALEXIS TEO PEI QI', value='Teo, Alexis')



df['NAME'] = df['NAME'].replace(regex=r'Enriquez, Roxanne Rose Zulueta', value='ROSE ZULUETA ENRIQUE, ROXANNE')
df['NAME'] = df['NAME'].replace(regex=r'ENRIQUEZ, ROXANNE ROSE ZULUETA', value='ROSE ZULUETA ENRIQUE, ROXANNE')
df['NAME'] = df['NAME'].replace(regex=r'ENRIQUEZ, ROSE ZULUETA, ROXANN', value='ROSE ZULUETA ENRIQUE, ROXANNE')

df['NAME'] = df['NAME'].replace(regex=r'Goh, Clara', value='GOH, CLARA')

df['NAME'] = df['NAME'].replace(regex=r'WONG JIN ROU', value='Wong, Jin Rou')



df['NAME'] = df['NAME'].replace(regex=r'THANA RAJAN, S/O, THIRUBEN', value='THANA RAJAN, THIRUBEN  S/O')
df['NAME'] = df['NAME'].replace(regex=r'Rajan Thiruben Thana', value='THANA RAJAN, THIRUBEN  S/O')


df['NAME'] = df['NAME'].replace(regex=r'Raam Kumar Muthukumaran', value='Muthukumaran, Raam Kumar')

df['NAME'] = df['NAME'].replace(regex=r'Chia Wei En, Shawn', value='Chia, Shawn')



df['NAME'] = df['NAME'].replace(regex=r'QUEK XIN YONG JEREMIAH', value='QUEK, JEREMIAH')
df['NAME'] = df['NAME'].replace(regex=r'quek xin yong jeremi', value='QUEK, JEREMIAH')


df['NAME'] = df['NAME'].replace(regex=r'Chiu, Jingwen', value='Chiu, Jing Wen')



df['NAME'] = df['NAME'].replace(regex=r'LEE YU FOONG (LI YUFENG)', value='Lee, Yu Foong')


df['NAME'] = df['NAME'].replace(regex=r'LOW CHUAN YI', value='Low, Chuan Yi')



df['NAME'] = df['NAME'].replace(regex=r'Rachel Yang Bing Jie', value='YANG, RACHEL')

df['NAME'] = df['NAME'].replace(regex=r'Jun Yu Low', value='Low, Jun Yu')
df['NAME'] = df['NAME'].replace(regex=r'Low Jun Yu', value='Low, Jun Yu')

df['NAME'] = df['NAME'].replace(regex=r'Dewey Ng', value='Ng, Dewey')
df['NAME'] = df['NAME'].replace(regex=r'NG, DEWEY', value='Ng, Dewey')

df['NAME'] = df['NAME'].replace(regex=r'., Brendon Ting Li King', value='Ting Li King, Brandon')
df['NAME'] = df['NAME'].replace(regex=r'TAN KIAN SWEE ISAAC', value='Tan, Isaac')
df['NAME'] = df['NAME'].replace(regex=r'TAN, KIAN SWEE ISAAC', value='Tan, Isaac')

df['NAME'] = df['NAME'].replace(regex=r'GAO SHENGWEI', value='Gao, Shengwei')
df['NAME'] = df['NAME'].replace(regex=r'NEO SHYH KAI', value='Neo, Shyh Kai')
df['NAME'] = df['NAME'].replace(regex=r'ANGEL LIM YU XIN', value='Lim, Yuxin Angel')
df['NAME'] = df['NAME'].replace(regex=r'LAUREL, LIM JIA EN', value='LAUREL LIM JIA EN')

df['NAME'] = df['NAME'].replace(regex=r'LOH DING RONG, ANSON', value='Loh, Anson')

df['NAME'] = df['NAME'].replace(regex=r'SEAN, TEE YU XIANG', value='SEAN TEE YU XIANG')
df['NAME'] = df['NAME'].replace(regex=r'NG JUN HENG, RYAN', value='Ng, Ryan')
df['NAME'] = df['NAME'].replace(regex=r'LIM WEE ENG', value='Lim, Wee Eng')


df['NAME'] = df['NAME'].replace(regex=r'Chen Xiang Ang', value='Ang, Chen Xiang')

df['NAME'] = df['NAME'].replace(regex=r'CHUA JE-AN, GARRETT', value='Chua, Garrett')
df['NAME'] = df['NAME'].replace(regex=r'Garrett Chua', value='Chua, Garrett')


df['NAME'] = df['NAME'].replace(regex=r'Looi, Yan Ning Samantha', value='LOOI YAN NING, SAMANTHA')
df['NAME'] = df['NAME'].replace(regex=r'Lim, Sheila Qi Le', value='Lim Qi Le, Sheila')
df['NAME'] = df['NAME'].replace(regex=r'NG, CHIN HUI', value='Ng, Chin Hui')
df['NAME'] = df['NAME'].replace(regex=r'SEAN, TEE YU XIANG', value='SEAN TEE YU XIANG')


df['NAME'] = df['NAME'].replace(regex=r'Edsel Poh', value='Poh, Edsel')
df['NAME'] = df['NAME'].replace(regex=r'Gautam, Kulandaiveludayar Se', value='Kulandaiveludayar Se, Gautam')



df['NAME'] = df['NAME'].replace(regex=r'Hanim, Asmah', value='Asmah, Hanim')
df['NAME'] = df['NAME'].replace(regex=r'YOON SAE YOUNG', value='Yoon, Sae Young')
df['NAME'] = df['NAME'].replace(regex=r'Wong, Alexis Li Xuan', value='Wong, Alexis')

df['NAME'] = df['NAME'].replace(regex=r'TAN JUN ERN KEAN', value='Tan Jun Ern Kean')

df['NAME'] = df['NAME'].replace(regex=r'JOEY LIM ZI YI', value='Lim, Joey')
df['NAME'] = df['NAME'].replace(regex=r'Bernie Chong Yi Shan', value='Chong, Bernie')

df['NAME'] = df['NAME'].replace(regex=r'Tan Ting Jun', value='Tan, Ting Jun')
df['NAME'] = df['NAME'].replace(regex=r'Xiao, Hongyang', value='Xiao, Hong Yang')
df['NAME'] = df['NAME'].replace(regex=r'Reiner Leong Rong Xi', value='Leong, Reiner')


df['NAME'] = df['NAME'].replace(regex=r'CHEN JING YING JANELLE', value='Chen, Janelle')
df['NAME'] = df['NAME'].replace(regex=r'Toh Wei De', value='Toh, Wei De')
#df['NAME'] = df['NAME'].replace(regex=r'SheikAluu\'udin\,\sSkeik\sFayz', value='Sheik\sAluu\'udin\,\sSkeik\sFayz')

df['NAME'] = df['NAME'].replace(regex=r'kapil, Arnav', value='Kapil, Arnav')
df['NAME'] = df['NAME'].replace(regex=r'KIRAN, SEBASTIAN JEHUNATHAN', value='Sebastian Jehunathan, Kiran')
df['NAME'] = df['NAME'].replace(regex=r'LAU, REINA', value='Lau, Reina')
df['NAME'] = df['NAME'].replace(regex=r'Lau Reina', value='Lau, Reina')


df['NAME'] = df['NAME'].replace(regex=r'Si Hui, Amanda, Loo', value='Amanda, Loo Si Hui')
df['NAME'] = df['NAME'].replace(regex=r'Zou Aolin q', value='Zou Aolin')
df['NAME'] = df['NAME'].replace(regex=r'Thang Ying Yue', value='Thang, Ying Yue')

df['NAME'] = df['NAME'].replace(regex=r'NEO EE , NIN', value='Neo, Ee Nin')
df['NAME'] = df['NAME'].replace(regex=r'Neo Ee Nin', value='Neo, Ee Nin')
df['NAME'] = df['NAME'].replace(regex=r'Emma Tan Yi Min', value='TAN, EMMA YI MIN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Emma Yi Min', value='TAN, EMMA YI MIN')

df['NAME'] = df['NAME'].replace(regex=r'WONG, JIN ROU', value='Wong, Jin Rou')
df['NAME'] = df['NAME'].replace(regex=r'Seah, Michelle Xin Yi', value='Seah, Michelle')
df['NAME'] = df['NAME'].replace(regex=r'Jing Wen, Chiu', value='Chiu, Jing Wen')
df['NAME'] = df['NAME'].replace(regex=r'NG, ROY', value='Ng Kai Leong, Roy')
df['NAME'] = df['NAME'].replace(regex=r'Ng, Kai Leong Roy', value='Ng Kai Leong, Roy')
df['NAME'] = df['NAME'].replace(regex=r'., Ashlee Ong Yuxi', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'Ashlee Ong Yuxi', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'Ong, Ashlee', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'Wee, Elliott', value='Wee, Elliot')
df['NAME'] = df['NAME'].replace(regex=r'., Cheong Yin Yern Jose', value='Joseph Cheong Yin Yern')
df['NAME'] = df['NAME'].replace(regex=r'Thang Ying Yue', value='Thang, Ying Yue')
df['NAME'] = df['NAME'].replace(regex=r'Hongyang, Xiao', value='XIAO, HONGYANG')
df['NAME'] = df['NAME'].replace(regex=r'Tan Shou Yi Rei (Chen Shouyi)', value='Tan Shou Yi Rei')
df['NAME'] = df['NAME'].replace(regex=r'Ong, Ashlee Yuxi Yuxi', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'Low, Jonathan', value='Low Jun Jie, Jonathan')
df['NAME'] = df['NAME'].replace(regex=r'Murali, Sham', value='Kumar Shanjie, Murali Sham')
df['NAME'] = df['NAME'].replace(regex=r'SHANJIE, MURALI SHAM KUMAR', value='Kumar Shanjie, Murali Sham')
df['NAME'] = df['NAME'].replace(regex=r'Koh, Aaron Xu Qing', value='Koh, Aaron')

df['NAME'] = df['NAME'].replace(regex=r'Tan Shou Yi Rei (Chen Shouyi)', value='Tan Shou Yi Rei')


df['NAME'] = df['NAME'].replace(regex=r'Mook, He Jun', value='HE, MOOK JUN')
df['NAME'] = df['NAME'].replace(regex=r'Leong, Lorraine', value='Leong Jia En, Lorraine')
df['NAME'] = df['NAME'].replace(regex=r'Joseph Cheong Yin Yern', value='Cheong, Joseph')
df['NAME'] = df['NAME'].replace(regex=r'Wee, Junn Elliott', value='Wee, Elliot')
df['NAME'] = df['NAME'].replace(regex=r'Sheik, Fayz', value='Sheik Alau\'ddin, Sheik Fayz')
df['NAME'] = df['NAME'].replace(regex=r'Keondre Liew Zhen Kai', value='Liew, Keondre Zhen Kai')




df['NAME'] = df['NAME'].replace(regex=r'TOH, TEDD', value='Toh Jun Xi, Tedd')



df['NAME'] = df['NAME'].replace(regex=r'Chuhan Zhong', value='ZHONG, CHUHAN')
df['NAME'] = df['NAME'].replace(regex=r'Lee, Daniel', value='Lee Zheng Xian, Daniel Christopher')
df['NAME'] = df['NAME'].replace(regex=r'GOH, YEN YOUNG, AMELIA', value='Goh Yen Young Amelia')
df['NAME'] = df['NAME'].replace(regex=r'Liew, Keondre Zhen Kai', value='LIEW, KEONDRE')
df['NAME'] = df['NAME'].replace(regex=r'Janelle, Lum', value='Lum Wai Yan, Janelle')


df['NAME'] = df['NAME'].replace(regex=r'Teh Ying Shan', value='Teh, Ying Shan')

df['NAME'] = df['NAME'].replace(regex=r'Mayah Elizabeth Kaufmann Q', value='Mayah Elizabeth Kaufmann')


df['NAME'] = df['NAME'].replace(regex=r'TNG, KAI XIN', value='Tng Kai Xin')
# Correct some of the known name variations
# Need to fix 2024 NSG names extracts (extracting Q/q with name)

df['NAME'] = df['NAME'].replace(regex=r'PRAHARSH, RYAN', value='S/O SUBASH SOMAN, PRAHARSH RYAN')
df['NAME'] = df['NAME'].replace(regex=r'Ryan, Praharsh', value='S/O SUBASH SOMAN, PRAHARSH RYAN')
df['NAME'] = df['NAME'].replace(regex=r'Ryan Praharsh', value='S/O SUBASH SOMAN, PRAHARSH RYAN')

df['NAME'] = df['NAME'].replace(regex=r'Ryan Praharsh', value='S/O SUBASH SOMAN, PRAHARSH RYAN')


df['NAME'] = df['NAME'].replace(regex=r'TAN, ELIZABETH ANN SHEE R', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Elizabeth Ann', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'TAN, SHEE RU, ELIZABETH-A', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Elizabeth-Ann Tan', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Elizabeth-Ann', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Tan Shee Ru, Elizabeth-Ann', value='TAN, ELIZABETH-ANN')

df['NAME'] = df['NAME'].replace(regex=r'Louis, Marc', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'LOUIS, MARC BRIAN', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'LOUIS, MARC BRIAN BRIAN', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'Louis, Marc Brian Brian', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'Marc Brian Louis', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'Brian louis, Marc', value='Louis, Marc Brian')




df['NAME'] = df['NAME'].replace(regex=r'TAN JUN JIE', value='Tan, Jun Jie')
df['NAME'] = df['NAME'].replace(regex=r'Tan Jun Jie', value='Tan, Jun Jie')

df['NAME'] = df['NAME'].replace(regex=r'Shanti Veronica Pereira', value='PEREIRA, VERONICA SHANTI')
df['NAME'] = df['NAME'].replace(regex=r'LIEW, YEE LING, BERNICE', value='Liew, Bernice')
df['NAME'] = df['NAME'].replace(regex=r'LIEW, BERNICE YEE LING', value='Liew, Bernice')

df['NAME'] = df['NAME'].replace(regex=r'Mark Lee', value='LEE, MARK REN')
df['NAME'] = df['NAME'].replace(regex=r'Lee, Mark', value='LEE, MARK REN')





df['NAME'] = df['NAME'].replace(regex=r'ANG, CHEN XIANG', value='Ang, Chen Xiang')



df['NAME'] = df['NAME'].replace(regex=r'LEE YU FOONG (LI YUFENG)', value='Lee, Yu Foong')
df['NAME'] = df['NAME'].replace(regex=r'YAP, SEAN YEE', value='Yap, Sean Yee')

df['NAME'] = df['NAME'].replace(regex=r'ALEXIS TEO PEI QI', value='Teo Pei Qi, Alexis')
df['NAME'] = df['NAME'].replace(regex=r'Alexis Teo Pei Qi', value='Teo Pei Qi, Alexis')
df['NAME'] = df['NAME'].replace(regex=r'Teo Pei Qi, Alexis Q', value='Teo Pei Qi, Alexis')




df['NAME'] = df['NAME'].replace(regex=r'Enriquez, Roxanne Rose Zulueta', value='ROSE ZULUETA ENRIQUE, ROXANNE')
df['NAME'] = df['NAME'].replace(regex=r'ENRIQUEZ, ROXANNE ROSE ZULUETA', value='ROSE ZULUETA ENRIQUE, ROXANNE')
df['NAME'] = df['NAME'].replace(regex=r'ENRIQUEZ, ROSE ZULUETA, ROXANN', value='ROSE ZULUETA ENRIQUE, ROXANNE')

df['NAME'] = df['NAME'].replace(regex=r'Goh, Clara', value='GOH, CLARA')

df['NAME'] = df['NAME'].replace(regex=r'WONG JIN ROU', value='Wong, Jin Rou')



df['NAME'] = df['NAME'].replace(regex=r'THANA RAJAN, S/O, THIRUBEN', value='THANA RAJAN, THIRUBEN  S/O')
df['NAME'] = df['NAME'].replace(regex=r'Rajan Thiruben Thana', value='THANA RAJAN, THIRUBEN  S/O')
df['NAME'] = df['NAME'].replace(regex=r'Thana Rajan, Thiruben', value='THANA RAJAN, THIRUBEN  S/O')



df['NAME'] = df['NAME'].replace(regex=r'Raam Kumar Muthukumaran', value='Muthukumaran, Raam Kumar')

df['NAME'] = df['NAME'].replace(regex=r'CHIA, WEI EN, SHAWN', value='Chia, Shawn')


df['NAME'] = df['NAME'].replace(regex=r'QUEK XIN YONG JEREMIAH', value='QUEK, JEREMIAH')
df['NAME'] = df['NAME'].replace(regex=r'quek xin yong jeremi', value='QUEK, JEREMIAH')


df['NAME'] = df['NAME'].replace(regex=r'Chiu, Jingwen', value='Chiu, Jing Wen')



df['NAME'] = df['NAME'].replace(regex=r'LEE YU FOONG (LI YUFENG)', value='Lee, Yu Foong')


df['NAME'] = df['NAME'].replace(regex=r'LOW CHUAN YI', value='Low, Chuan Yi')

df['NAME'] = df['NAME'].replace(regex=r'Rachel Yang Bing Jie', value='YANG, RACHEL')


df['NAME'] = df['NAME'].replace(regex=r'Dewey Ng', value='Ng, Dewey')
df['NAME'] = df['NAME'].replace(regex=r'NG, DEWEY', value='Ng, Dewey')

df['NAME'] = df['NAME'].replace(regex=r'., Brendon Ting Li King', value='Ting Li King, Brandon')
df['NAME'] = df['NAME'].replace(regex=r'TAN KIAN SWEE ISAAC', value='Tan, Isaac')
df['NAME'] = df['NAME'].replace(regex=r'TAN, KIAN SWEE ISAAC', value='Tan, Isaac')

df['NAME'] = df['NAME'].replace(regex=r'GAO SHENGWEI', value='Gao, Shengwei')
df['NAME'] = df['NAME'].replace(regex=r'NEO SHYH KAI', value='Neo, Shyh Kai')
df['NAME'] = df['NAME'].replace(regex=r'ANGEL LIM YU XIN', value='Lim, Yuxin Angel')
df['NAME'] = df['NAME'].replace(regex=r'LAUREL, LIM JIA EN', value='LAUREL LIM JIA EN')

df['NAME'] = df['NAME'].replace(regex=r'LOH DING RONG, ANSON', value='Loh, Anson')

df['NAME'] = df['NAME'].replace(regex=r'SEAN, TEE YU XIANG', value='SEAN TEE YU XIANG')
df['NAME'] = df['NAME'].replace(regex=r'NG JUN HENG, RYAN', value='Ng, Ryan')
df['NAME'] = df['NAME'].replace(regex=r'LIM WEE ENG', value='Lim, Wee Eng')


df['NAME'] = df['NAME'].replace(regex=r'Chen Xiang Ang', value='Ang, Chen Xiang')

df['NAME'] = df['NAME'].replace(regex=r'CHUA JE-AN, GARRETT', value='Chua, Garrett')
df['NAME'] = df['NAME'].replace(regex=r'Garrett Chua', value='Chua, Garrett')


df['NAME'] = df['NAME'].replace(regex=r'Looi, Yan Ning Samantha', value='LOOI YAN NING, SAMANTHA')
df['NAME'] = df['NAME'].replace(regex=r'Lim, Sheila Qi Le', value='Lim Qi Le, Sheila')
df['NAME'] = df['NAME'].replace(regex=r'NG, CHIN HUI', value='Ng, Chin Hui')
df['NAME'] = df['NAME'].replace(regex=r'SEAN, TEE YU XIANG', value='SEAN TEE YU XIANG')

df['NAME'] = df['NAME'].replace(regex=r'CHUA HSIN-WEN CLARA', value='Chua Hsin-Wen, Clara')
df['NAME'] = df['NAME'].replace(regex=r'Chua Hsin-Wen Clara', value='Chua Hsin-Wen, Clara')
df['NAME'] = df['NAME'].replace(regex=r'Chua Hsin-Wen Clara Q', value='Chua Hsin-Wen, Clara')
df['NAME'] = df['NAME'].replace(regex=r'HSIN-WEN CLARA, Chua', value='Chua Hsin-Wen, Clara')
df['NAME'] = df['NAME'].replace(regex=r'CHUA HSIN-WEN CLARA', value='Chua Hsin-Wen, Clara')
df['NAME'] = df['NAME'].replace(regex=r'Chua, Clara Q', value='Chua Hsin-Wen, Clara')


df['NAME'] = df['NAME'].replace(regex=r'Edsel Poh', value='Poh, Edsel')
df['NAME'] = df['NAME'].replace(regex=r'Gautam, Kulandaiveludayar Se', value='Kulandaiveludayar Se, Gautam')


df['NAME'] = df['NAME'].replace(regex=r'Hanim, Asmah', value='Asmah, Hanim')
df['NAME'] = df['NAME'].replace(regex=r'YOON SAE YOUNG', value='Yoon, Sae Young')
df['NAME'] = df['NAME'].replace(regex=r'Wong, Alexis Li Xuan', value='Wong, Alexis')

df['NAME'] = df['NAME'].replace(regex=r'TAN JUN ERN KEAN', value='Tan Jun Ern Kean')

df['NAME'] = df['NAME'].replace(regex=r'JOEY LIM ZI YI', value='Lim, Joey')
df['NAME'] = df['NAME'].replace(regex=r'Bernie Chong Yi Shan', value='Chong, Bernie')

df['NAME'] = df['NAME'].replace(regex=r'Tan Ting Jun', value='Tan, Ting Jun')
df['NAME'] = df['NAME'].replace(regex=r'Xiao, Hongyang', value='Xiao, Hong Yang')
df['NAME'] = df['NAME'].replace(regex=r'Reiner Leong Rong Xi', value='Leong, Reiner')


df['NAME'] = df['NAME'].replace(regex=r'CHEN JING YING JANELLE', value='Chen, Janelle')
df['NAME'] = df['NAME'].replace(regex=r'Toh Wei De', value='Toh, Wei De')
#df['NAME'] = df['NAME'].replace(regex=r'SheikAluu\'udin\,\sSkeik\sFayz', value='Sheik\sAluu\'udin\,\sSkeik\sFayz')

df['NAME'] = df['NAME'].replace(regex=r'kapil, Arnav', value='Kapil, Arnav')
df['NAME'] = df['NAME'].replace(regex=r'KIRAN, SEBASTIAN JEHUNATHAN', value='Sebastian Jehunathan, Kiran')
df['NAME'] = df['NAME'].replace(regex=r'LAU, REINA', value='Lau, Reina')
df['NAME'] = df['NAME'].replace(regex=r'Lau Reina', value='Lau, Reina')


df['NAME'] = df['NAME'].replace(regex=r'Zou Aolin q', value='Zou Aolin')
df['NAME'] = df['NAME'].replace(regex=r'Thang Ying Yue', value='Thang, Ying Yue')

df['NAME'] = df['NAME'].replace(regex=r'NEO EE , NIN', value='Neo, Ee Nin')
df['NAME'] = df['NAME'].replace(regex=r'Neo Ee Nin', value='Neo, Ee Nin')
df['NAME'] = df['NAME'].replace(regex=r'Emma Tan Yi Min', value='TAN, EMMA YI MIN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Emma Yi Min', value='TAN, EMMA YI MIN')


df['NAME'] = df['NAME'].replace(regex=r'Si Hui, Amanda, Loo', value='Amanda, Loo Si Hui')
df['NAME'] = df['NAME'].replace(regex=r'Si Hui Amanda, Loo', value='Amanda, Loo Si Hui')



df['NAME'] = df['NAME'].replace(regex=r'WONG, JIN ROU', value='Wong, Jin Rou')
df['NAME'] = df['NAME'].replace(regex=r'Seah, Michelle Xin Yi', value='Seah, Michelle')
df['NAME'] = df['NAME'].replace(regex=r'Jing Wen, Chiu', value='Chiu, Jing Wen')
df['NAME'] = df['NAME'].replace(regex=r'NG, ROY', value='Ng Kai Leong, Roy')
df['NAME'] = df['NAME'].replace(regex=r'Ng, Kai Leong Roy', value='Ng Kai Leong, Roy')
df['NAME'] = df['NAME'].replace(regex=r'., Ashlee Ong Yuxi', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'Ashlee Ong Yuxi', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'Ong, Ashlee', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'Wee, Elliott', value='Wee, Elliot')
df['NAME'] = df['NAME'].replace(regex=r'., Cheong Yin Yern Jose', value='Joseph Cheong Yin Yern')
df['NAME'] = df['NAME'].replace(regex=r'Thang Ying Yue', value='Thang, Ying Yue')
df['NAME'] = df['NAME'].replace(regex=r'Hongyang, Xiao', value='XIAO, HONGYANG')
df['NAME'] = df['NAME'].replace(regex=r'Tan Shou Yi Rei (Chen Shouyi)', value='Tan Shou Yi Rei')
df['NAME'] = df['NAME'].replace(regex=r'Ong, Ashlee Yuxi Yuxi', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'Low, Jonathan', value='Low Jun Jie, Jonathan')
df['NAME'] = df['NAME'].replace(regex=r'Murali, Sham', value='Kumar Shanjie, Murali Sham')
df['NAME'] = df['NAME'].replace(regex=r'SHANJIE, MURALI SHAM KUMAR', value='Kumar Shanjie, Murali Sham')
df['NAME'] = df['NAME'].replace(regex=r'Koh, Aaron Xu Qing', value='Koh, Aaron')

df['NAME'] = df['NAME'].replace(regex=r'^Tan\sShou\sYi\sRei$', value='Tan Shou Yi Rei (Chen Shouyi)')



df['NAME'] = df['NAME'].replace(regex=r'KUMAR, VIRESH', value='Kumar, Viresh')
df['NAME'] = df['NAME'].replace(regex=r'KUMAR, S VIRESH', value='Kumar, Viresh')
df['NAME'] = df['NAME'].replace(regex=r's Viresh Kumar', value='Kumar, Viresh')
df['NAME'] = df['NAME'].replace(regex=r'Kumar, S Viresh', value='Kumar, Viresh')


df['NAME'] = df['NAME'].replace(regex=r'Mook, He Jun', value='HE, MOOK JUN')
df['NAME'] = df['NAME'].replace(regex=r'Leong, Lorraine', value='Leong Jia En, Lorraine')
df['NAME'] = df['NAME'].replace(regex=r'Joseph Cheong Yin Yern', value='Cheong, Joseph')
df['NAME'] = df['NAME'].replace(regex=r'Wee, Junn Elliott', value='Wee, Elliot')
df['NAME'] = df['NAME'].replace(regex=r'Sheik, Fayz', value='Sheik Alau\'ddin, Sheik Fayz')
df['NAME'] = df['NAME'].replace(regex=r'Keondre Liew Zhen Kai', value='Liew, Keondre Zhen Kai')




df['NAME'] = df['NAME'].replace(regex=r'TOH, TEDD', value='Toh Jun Xi, Tedd')
df['NAME'] = df['NAME'].replace(regex=r'Toh, Tedd', value='Toh Jun Xi, Tedd')





df['NAME'] = df['NAME'].replace(regex=r'Chuhan Zhong', value='ZHONG, CHUHAN')
df['NAME'] = df['NAME'].replace(regex=r'Lee, Daniel', value='Lee Zheng Xian, Daniel Christopher')



df['NAME'] = df['NAME'].replace(regex=r'GOH, YEN YOUNG, AMELIA', value='Goh Yen Young Amelia')
df['NAME'] = df['NAME'].replace(regex=r'lim, eiffer', value='LIM YING QI, EIFFER')
df['NAME'] = df['NAME'].replace(regex=r'Eiffer, Lim Ying Qi', value='LIM YING QI, EIFFER')
df['NAME'] = df['NAME'].replace(regex=r'Liew, Keondre Zhen Kai', value='LIEW, KEONDRE')
df['NAME'] = df['NAME'].replace(regex=r'Janelle, Lum', value='Lum Wai Yan, Janelle')
df['NAME'] = df['NAME'].replace(regex=r'Teh Ying Shan', value='Teh, Ying Shan')
df['NAME'] = df['NAME'].replace(regex=r'Ying Shan, Teh', value='Teh, Ying Shan')


df['NAME'] = df['NAME'].replace(regex=r'Mayah Elizabeth Kaufmann Q', value='Mayah Elizabeth Kaufmann')
df['NAME'] = df['NAME'].replace(regex=r'TNG, KAI XIN', value='Tng Kai Xin')
df['NAME'] = df['NAME'].replace(regex=r'YANG, BOBBI', value='Bobbi Victoria Yang Yen (Weng Xuan)')

df['NAME'] = df['NAME'].replace(regex=r'Ng, Si En Tabitha', value='Ng Si En, Tabitha')
df['NAME'] = df['NAME'].replace(regex=r'Hoh Ying Min', value='Hoh, Ying Min')
df['NAME'] = df['NAME'].replace(regex=r'Tedd Toh', value='Toh Jun Xi, Tedd')
df['NAME'] = df['NAME'].replace(regex=r'Irfan Qabeel Mohamman', value='Mohammad Irfan, Qabeel')
df['NAME'] = df['NAME'].replace(regex=r'Sebastian, Kiran', value='Sebastian Jehunathan, Kiran')
df['NAME'] = df['NAME'].replace(regex=r'TANG, CAYMAN', value='Tang Kai Sheng, Cayman')
df['NAME'] = df['NAME'].replace(regex=r'ZHONG, CHUHAN', value='Zhong, Chuhan')

df['NAME'] = df['NAME'].replace(regex=r'Reuben Lee Siong En', value='LEE SIONG EN, REUBEN RAINER')
df['NAME'] = df['NAME'].replace(regex=r'Tam, Joash', value='Tam Jong-Hng, Joash')
df['NAME'] = df['NAME'].replace(regex=r'Pooran Sankaresh Dharmaraj', value='Pooran, Dharmaraj')

df['NAME'] = df['NAME'].replace(regex=r'Meena, Girish Chandramohan', value='Chandramohan Meena, Girish')
df['NAME'] = df['NAME'].replace(regex=r'Wong Jin Rou', value='Wong, Jin Rou')
df['NAME'] = df['NAME'].replace(regex=r'MOHHAMMAD IRFAN, QABEEL', value='Mohammad Irfan, Qabeel')
df['NAME'] = df['NAME'].replace(regex=r'BIN MD, MOHAMMAD IRFAN QABEE', value='Mohammad Irfan, Qabeel')



df['NAME'] = df['NAME'].replace(regex=r'TAM, JOASH', value='Tam Jong-Hng, Joash')
df['NAME'] = df['NAME'].replace(regex=r'Verboon Lara', value='Verboon, Lara')
df['NAME'] = df['NAME'].replace(regex=r'Verboon , Lara', value='Verboon, Lara')

df['NAME'] = df['NAME'].replace(regex=r'Megan Anne Ying Ka Mun Q', value='Megan Anne Ying Ka Mun')
df['NAME'] = df['NAME'].replace(regex=r'Woong, Hui Min Rachel', value='Woong, Rachel Hui Min')

df['NAME'] = df['NAME'].replace(regex=r'FENG, HAN LIN', value='Feng, Han Lin')
df['NAME'] = df['NAME'].replace(regex=r'Angela Tan Shi Jin', value='TAN, ANGELA')
df['NAME'] = df['NAME'].replace(regex=r'Zhang, Ruiheng, Zayden', value='Zhang, Zayden')



df['NAME'] = df['NAME'].replace(regex=r'Norhisham, Jamie El Redha Ang El Redha Ang', value='Jamie El-Red Ang')
df['NAME'] = df['NAME'].replace(regex=r'Jamie Norhisham', value='Jamie El-Red Ang')
df['NAME'] = df['NAME'].replace(regex=r'Norhisham, Jamie', value='Jamie El-Red Ang')

### 2024###

df['NAME'] = df['NAME'].replace(regex=r'D/O JAIGANTH, LAAVINIA', value='JAIGANTH, LAAVINIA')
df['NAME'] = df['NAME'].replace(regex=r'Jaiganth, Laavinia', value='JAIGANTH, LAAVINIA')
df['NAME'] = df['NAME'].replace(regex=r'Laavina Jaganith', value='JAIGANTH, LAAVINIA')


df['NAME'] = df['NAME'].replace(regex=r'Faith Zhen Ford', value='FAITH ZHEN FORD')
df['NAME'] = df['NAME'].replace(regex=r'Ford, Faith', value='FAITH ZHEN FORD')
df['NAME'] = df['NAME'].replace(regex=r'FORD, FAITH', value='FAITH ZHEN FORD')
df['NAME'] = df['NAME'].replace(regex=r'Ford Faith', value='FAITH ZHEN FORD')

df['NAME'] = df['NAME'].replace(regex=r'Koh Shun Yi Audrey', value='Koh, Shun Yi Audrey')
df['NAME'] = df['NAME'].replace(regex=r'Koh Shun Yi Audrey Q', value='Koh, Shun Yi Audrey')
df['NAME'] = df['NAME'].replace(regex=r'Shun Yi Audrey Koh', value='Koh, Shun Yi Audrey')
df['NAME'] = df['NAME'].replace(regex=r'KOH SHUN YI AUDREY', value='Koh, Shun Yi Audrey')



df['NAME'] = df['NAME'].replace(regex=r'Romaine Soh', value='ROMAINE SOH RUI MIN')
df['NAME'] = df['NAME'].replace(regex=r'Soh, Romaine', value='ROMAINE SOH RUI MIN')
df['NAME'] = df['NAME'].replace(regex=r'Soh, Rui Min Romaine', value='ROMAINE SOH RUI MIN')

df['NAME'] = df['NAME'].replace(regex=r'khoo, Tyeisha rene misson', value='Khoo, Tyeisha')

df['NAME'] = df['NAME'].replace(regex=r'Faith Jia Yin Koh Q', value='Koh, Faith Jia Yin')
df['NAME'] = df['NAME'].replace(regex=r'FAITH JIA YIN KOH', value='Koh, Faith Jia Yin')
df['NAME'] = df['NAME'].replace(regex=r'Faith Jia Yin Koh', value='Koh, Faith Jia Yin')
df['NAME'] = df['NAME'].replace(regex=r'Koh, Faith', value='Koh, Faith Jia Yin')

df['NAME'] = df['NAME'].replace(regex=r'LIM, EIFFER YING QI', value='LIM YING QI, EIFFER')
df['NAME'] = df['NAME'].replace(regex=r'lim, eiffer', value='LIM YING QI, EIFFER')
df['NAME'] = df['NAME'].replace(regex=r'Eiffer, Lim Ying Qi', value='LIM YING QI, EIFFER')

df['NAME'] = df['NAME'].replace(regex=r'Chui Ling Goh', value='GOH, CHUI LING')

df['NAME'] = df['NAME'].replace(regex=r'Chong Le Xuan, Chloe', value='CHONG LE XUAN, CHLOE')
df['NAME'] = df['NAME'].replace(regex=r'CHONG, LE XUAN, CHLOE', value='CHONG LE XUAN, CHLOE')
df['NAME'] = df['NAME'].replace(regex=r'CHONG, CHLOE LE XUAN', value='CHONG LE XUAN, CHLOE')



df['NAME'] = df['NAME'].replace(regex=r'SIDHOM, MANUELA', value='Sidhom, Manuela')
df['NAME'] = df['NAME'].replace(regex=r'Manuela Sidhom Q', value='Sidhom, Manuela')
df['NAME'] = df['NAME'].replace(regex=r'Manuela Sidhom q', value='Sidhom, Manuela')
df['NAME'] = df['NAME'].replace(regex=r'Manuela Sidhom', value='Sidhom, Manuela')

df['NAME'] = df['NAME'].replace(regex=r'^Gan,\sSteffi$', value='Gan, Steffi Kai Lin')
df['NAME'] = df['NAME'].replace(regex=r'Gan Kai-lin, Steffi', value='Gan, Steffi Kai Lin')




df['NAME'] = df['NAME'].replace(regex=r'Yu En Bernice, Tan', value='Tan Yu En, Bernice')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Yu En Bernice', value='Tan Yu En, Bernice')
#df['NAME'] = df['NAME'].replace(regex=r'Tan, Bernice', value='Tan Yu En, Bernice')
df['NAME'] = df['NAME'].replace(regex=r'Tan Yu En Bernice q', value='Tan Yu En, Bernice')
df['NAME'] = df['NAME'].replace(regex=r'Tan Yu En Bernice Q', value='Tan Yu En, Bernice')
#df['NAME'] = np.where(((df['NAME']=='Tan, Bernice')&(df['DOB']=='2010-10-05')), 'Tan Yu En, Bernice', 'Tan, Bernice')
df['NAME'] = df['NAME'].replace(regex=r'TAN, YUN RU BERNICE', value='Tan Yun Ru, Bernice')

#df['NAME'] = np.where(((df['NAME']=='Tan, Bernice')&(df['DOB']=='2000-11-16')), 'Tan Yun Ru, Bernice', 'Tan, Bernice')

df['NAME'] = df['NAME'].replace(regex=r'Ng, Yew Cheo', value='NG, YEW CHEO')

df['NAME'] = df['NAME'].replace(regex=r'Caelyn Chew Kai Ling Q', value='Chew Kai Ling, Caelyn')
df['NAME'] = df['NAME'].replace(regex=r'Caelyn Chew Kai Ling', value='Chew Kai Ling, Caelyn')

df['NAME'] = df['NAME'].replace(regex=r'FOO, NATALIE ZHI NING', value='Foo Zhi Ning, Natalie')
df['NAME'] = df['NAME'].replace(regex=r'Foo, Natalie', value='Foo Zhi Ning, Natalie')


df['NAME'] = df['NAME'].replace(regex=r'Xiu Yi, Leong', value='Leong, Xiu Yi')


df['NAME'] = df['NAME'].replace(regex=r'Shing Ling Goh', value='GOH SHING LING')
df['NAME'] = df['NAME'].replace(regex=r'Goh, Shing Ling', value='GOH SHING LING')



df['NAME'] = df['NAME'].replace(regex=r'Tia Louise Rozario', value='ROZARIO, TIA LOUISE')
df['NAME'] = df['NAME'].replace(regex=r'Rozario, Tia Louise', value='ROZARIO, TIA LOUISE')


df['NAME'] = df['NAME'].replace(regex=r'Chloe Chee En-Ya', value='CHLOE CHEE EN-YA')
df['NAME'] = df['NAME'].replace(regex=r'En-ya, Chloe', value='CHLOE CHEE EN-YA')
df['NAME'] = df['NAME'].replace(regex=r'Chee Chee En-ya, Chloe', value='CHLOE CHEE EN-YA')
df['NAME'] = df['NAME'].replace(regex=r'Chloe Chee En-Ya Q', value='CHLOE CHEE EN-YA')
df['NAME'] = df['NAME'].replace(regex=r'En-ya, Chloe', value='CHLOE CHEE EN-YA')
df['NAME'] = df['NAME'].replace(regex=r'Chee En-ya, Chloe', value='CHLOE CHEE EN-YA')


df['NAME'] = df['NAME'].replace(regex=r'Chuhan Zhong', value='Zhong, Chuhan')
df['NAME'] = df['NAME'].replace(regex=r'Megan Anne Ying Ka Mun Q', value='Zhong,')


df['NAME'] = df['NAME'].replace(regex=r'Lim Yee Chern Clara Q', value='LIM YEE CHERN CLARA')
df['NAME'] = df['NAME'].replace(regex=r'LIM, YEE CHERN CLARA', value='LIM YEE CHERN CLARA')
df['NAME'] = df['NAME'].replace(regex=r'Clara Lim Yee Chern', value='LIM YEE CHERN CLARA')
df['NAME'] = df['NAME'].replace(regex=r'Lim, Clara', value='LIM YEE CHERN CLARA')
df['NAME'] = df['NAME'].replace(regex=r'Lim Yee Chern Clara', value='LIM YEE CHERN CLARA')
df['NAME'] = df['NAME'].replace(regex=r'YEE CHERN CLARA LIM', value='LIM YEE CHERN CLARA')


df['NAME'] = df['NAME'].replace(regex=r'CHEW, JADE', value='CHEW YIU TSE JADE')
df['NAME'] = df['NAME'].replace(regex=r'Jade, Chew Yiu Tse', value='CHEW YIU TSE JADE')
df['NAME'] = df['NAME'].replace(regex=r'Chew, Jade', value='CHEW YIU TSE JADE')
df['NAME'] = df['NAME'].replace(regex=r'CHEW, YIU TSE JADE', value='CHEW YIU TSE JADE')
df['NAME'] = df['NAME'].replace(regex=r'Chew, Yiu Tse Jade', value='CHEW YIU TSE JADE')

df['NAME'] = df['NAME'].replace(regex=r'Tay, Esther SHEE WEI', value='TAY, ESTHER')
df['NAME'] = df['NAME'].replace(regex=r'Esther Tay', value='TAY, ESTHER')
df['NAME'] = df['NAME'].replace(regex=r'TAY, ESTHER SHEE WEI', value='TAY, ESTHER')
df['NAME'] = df['NAME'].replace(regex=r'TAY, ESTHER', value='TAY, ESTHER')
df['NAME'] = df['NAME'].replace(regex=r'Tay, Esther', value='TAY, ESTHER')
df['NAME'] = df['NAME'].replace(regex=r'ESTHER TAY SHEE WEI', value='TAY, ESTHER')



df['NAME'] = df['NAME'].replace(regex=r'Han Qi Feng', value='FENG, HAN QI')
df['NAME'] = df['NAME'].replace(regex=r'FENG HAN QI', value='FENG, HAN QI')
df['NAME'] = df['NAME'].replace(regex=r'Han Qi Feng', value='FENG, HAN QI')
df['NAME'] = df['NAME'].replace(regex=r'Feng Han Qi', value='FENG, HAN QI')
df['NAME'] = df['NAME'].replace(regex=r'Feng, Han Qi', value='FENG, HAN QI')

df['NAME'] = df['NAME'].replace(regex=r'Syed Ahmed Riadh, Sharifah Falisha', value='SYED AHMED RIADH, SHARIFAH FALISHA')
df['NAME'] = df['NAME'].replace(regex=r'Sharifah Falisha Binte Syed Ahmed', value='SYED AHMED RIADH, SHARIFAH FALISHA')

df['NAME'] = df['NAME'].replace(regex=r'Sam, Vanessa', value='SAM, VANESSA')

df['NAME'] = df['NAME'].replace(regex=r'Vanessa Lee Ying Zhuang', value='LEE, VANESSA')
df['NAME'] = df['NAME'].replace(regex=r'Lee Ying Zhuang, Vanessa', value='LEE, VANESSA')
df['NAME'] = df['NAME'].replace(regex=r'Lee, Vanessa', value='LEE, VANESSA')


df['NAME'] = df['NAME'].replace(regex=r'SNG, MICHELLE', value='MICHELLE SNG')
df['NAME'] = df['NAME'].replace(regex=r'SNG, SUAT LI, MICHELLE', value='MICHELLE SNG')
df['NAME'] = df['NAME'].replace(regex=r'Michelle Sng Suat Li', value='MICHELLE SNG')
df['NAME'] = df['NAME'].replace(regex=r'Sng, Michelle', value='MICHELLE SNG')
df['NAME'] = df['NAME'].replace(regex=r'Michelle Sng Suat Li', value='MICHELLE SNG')

df['NAME'] = df['NAME'].replace(regex=r'Rachel See Boon Lay', value='Rachel See')

df['NAME'] = df['NAME'].replace(regex=r'Yee, Wai Teng Melissa', value='YEE, MELISSA')
df['NAME'] = df['NAME'].replace(regex=r'YEE WAI TENG, MELISSA', value='YEE, MELISSA')

df['NAME'] = df['NAME'].replace(regex=r'Kerstin Ong Jing Rong', value='ONG, KERSTIN')
df['NAME'] = df['NAME'].replace(regex=r'Ong, Kerstin', value='ONG, KERSTIN')


df['NAME'] = df['NAME'].replace(regex=r'Chua, Joshua', value='CHUA, JOSHUA')

df['NAME'] = df['NAME'].replace(regex=r'Raphael, Ryan', value='RAPHAEL, RYAN')
df['NAME'] = df['NAME'].replace(regex=r'NG, RYAN RAPHAEL', value='RAPHAEL, RYAN')

df['NAME'] = df['NAME'].replace(regex=r'Xander Ho Ann Heng', value='HO ANN HENG, XANDER')
df['NAME'] = df['NAME'].replace(regex=r'Ho, Xander Ann Heng', value='HO ANN HENG, XANDER')
df['NAME'] = df['NAME'].replace(regex=r'HO, XANDER, ANN HENG', value='HO ANN HENG, XANDER')
df['NAME'] = df['NAME'].replace(regex=r'XANDER, HO ANN HENG', value='HO ANN HENG, XANDER')
df['NAME'] = df['NAME'].replace(regex=r'Ho Ann Heng, Xander', value='HO ANN HENG, XANDER')

df['NAME'] = df['NAME'].replace(regex=r'Tan, Tate', value='TATE TAN FUNG')

df['NAME'] = df['NAME'].replace(regex=r'Tan Hong An, Daryl', value='TAN, DARYL')
df['NAME'] = df['NAME'].replace(regex=r'Tan Daryl', value='TAN, DARYL')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Daryl', value='TAN, DARYL')

df['NAME'] = df['NAME'].replace(regex=r'Ong Ying Tat', value='ONG, YING TAT')
df['NAME'] = df['NAME'].replace(regex=r'Ong Ying Tat Q', value='ONG, YING TAT')

df['NAME'] = df['NAME'].replace(regex=r'S/O SUBASH SOMAN, PRAHARSH RYAN', value='RYAN, PRAHARSH')
df['NAME'] = df['NAME'].replace(regex=r'Ryan, Praharsh', value='RYAN, PRAHARSH')
df['NAME'] = df['NAME'].replace(regex=r'Ryan Praharsh', value='RYAN, PRAHARSH')
df['NAME'] = df['NAME'].replace(regex=r'PRAHARSH, RYAN', value='RYAN, PRAHARSH')

df['NAME'] = df['NAME'].replace(regex=r'LIM, CHUN KHAI, JON', value='LIM, JON')
df['NAME'] = df['NAME'].replace(regex=r'Lim, Jon', value='LIM, JON')

df['NAME'] = df['NAME'].replace(regex=r'Wei Guan, Chong', value='CHONG, WEI GUAN')
df['NAME'] = df['NAME'].replace(regex=r'Chong, Wei Guan', value='CHONG, WEI GUAN')


df['NAME'] = df['NAME'].replace(regex=r'Mohamed, Haja Fayiz', value='FAYIZ, MOHAMED HAJA')
df['NAME'] = df['NAME'].replace(regex=r'Osman, Amir Rusyaidi', value='OSMAN, AMIR RUSYAIDI')

df['NAME'] = df['NAME'].replace(regex=r'Jing Heng Ng', value='NG, JING HENG')
df['NAME'] = df['NAME'].replace(regex=r'Ng, Jing Heng', value='NG, JING HENG')


df['NAME'] = df['NAME'].replace(regex=r'Tan, Jonathan', value='TAN, JONATHAN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Jonathan Tan', value='TAN, JONATHAN')
df['NAME'] = df['NAME'].replace(regex=r'JONATHAN TAN', value='TAN, JONATHAN')



df['NAME'] = df['NAME'].replace(regex=r'Lucas Le Cong Fun', value='FUN LE CONG LUCAS')
df['NAME'] = df['NAME'].replace(regex=r'Fun Le Cong, Lucas', value='FUN LE CONG LUCAS')
df['NAME'] = df['NAME'].replace(regex=r'Fun Le Cong, Lucas Q', value='FUN LE CONG LUCAS')
df['NAME'] = df['NAME'].replace(regex=r'Fun, Le Cong Lucas', value='FUN LE CONG LUCAS')
df['NAME'] = df['NAME'].replace(regex=r'FUN, LE CONG LUCAS', value='FUN LE CONG LUCAS')


df['NAME'] = df['NAME'].replace(regex=r'PERCY MUNCHERJI, ZUBIN', value='ZUBIN PERCY MUNCHERJI')
df['NAME'] = df['NAME'].replace(regex=r'Muncherji, Zubin', value='ZUBIN PERCY MUNCHERJI')
df['NAME'] = df['NAME'].replace(regex=r'MUNCHERJI, ZUBIN PERCY', value='ZUBIN PERCY MUNCHERJI')
df['NAME'] = df['NAME'].replace(regex=r'Zubin Percy Muncherji', value='ZUBIN PERCY MUNCHERJI')

df['NAME'] = df['NAME'].replace(regex=r'Quek, Jun Jie Calvin', value='QUEK, JUN JIE CALVIN')
df['NAME'] = df['NAME'].replace(regex=r'Jun Jie Calvin Quek', value='QUEK, JUN JIE CALVIN')

df['NAME'] = df['NAME'].replace(regex=r'Chia, Shawn', value='SHAWN CHIA WEI EN')
df['NAME'] = df['NAME'].replace(regex=r'CHIA, WEI EN, SHAWN', value='SHAWN CHIA WEI EN')
df['NAME'] = df['NAME'].replace(regex=r'Chia Wei En, Shawn', value='SHAWN CHIA WEI EN')
df['NAME'] = df['NAME'].replace(regex=r'Wei En Shawn, Chia', value='SHAWN CHIA WEI EN')
df['NAME'] = df['NAME'].replace(regex=r'Shawn Chia Wei En', value='SHAWN CHIA WEI EN')
df['NAME'] = df['NAME'].replace(regex=r'CHIA, SHAWN WEI EN', value='SHAWN CHIA WEI EN')


df['NAME'] = df['NAME'].replace(regex=r'Jie Cong Jayden, Tan', value='JIE CONG JAYDEN, TAN')
df['NAME'] = df['NAME'].replace(regex=r'TAN , JIE CONG, JAYDEN', value='JIE CONG JAYDEN, TAN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Jayden', value='JIE CONG JAYDEN, TAN')

df['NAME'] = df['NAME'].replace(regex=r'Lim, Oliver', value='LIM, OLIVER')

df['NAME'] = df['NAME'].replace(regex=r'YEE, ERIC CHUN WAI', value='YEE, CHUN WAI, ERIC')
df['NAME'] = df['NAME'].replace(regex=r'Yee, Eric', value='YEE, CHUN WAI, ERIC')
df['NAME'] = df['NAME'].replace(regex=r'YEE CHUN WAI, ERIC', value='YEE, CHUN WAI, ERIC')
df['NAME'] = df['NAME'].replace(regex=r'Eric Yee Chun Wai', value='YEE, CHUN WAI, ERIC')
df['NAME'] = df['NAME'].replace(regex=r'Yee Chun Wai, Eric', value='YEE, CHUN WAI, ERIC')
df['NAME'] = df['NAME'].replace(regex=r'Yee, Chun Wai, Eric', value='YEE, CHUN WAI, ERIC')
df['NAME'] = df['NAME'].replace(regex=r'Yee Chun Wai, Eric', value='YEE, CHUN WAI, ERIC')
df['NAME'] = df['NAME'].replace(regex=r'YEE, ERIC CHUN WAI', value='YEE, CHUN WAI, ERIC')


df['NAME'] = df['NAME'].replace(regex=r'Rui Yong Soh', value='SOH RUI YONG GUILLAUME')

df['NAME'] = df['NAME'].replace(regex=r'TAN, AARON JUSTIN', value='TAN, AARON JUSTIN WEN JIE')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Aaron Justin', value='TAN, AARON JUSTIN WEN JIE')


df['NAME'] = df['NAME'].replace(regex=r'Shaun, Goh', value='GOH, SHAUN')
df['NAME'] = df['NAME'].replace(regex=r'Goh, Shaun', value='GOH, SHAUN')
df['NAME'] = df['NAME'].replace(regex=r'Shaun Goh', value='GOH, SHAUN')

df['NAME'] = df['NAME'].replace(regex=r'Ethan Yan', value='YAN, ETHAN')

df['NAME'] = df['NAME'].replace(regex=r'Brandon Heng Fu Hong', value='HENG, BRANDON')
df['NAME'] = df['NAME'].replace(regex=r'Heng, Brandon', value='HENG, BRANDON')
df['NAME'] = df['NAME'].replace(regex=r'Brandon Heng Fu Hong', value='HENG, BRANDON')
df['NAME'] = df['NAME'].replace(regex=r'HENG, BRANDON', value='HENG, BRANDON')
df['NAME'] = df['NAME'].replace(regex=r'HENG, FU HONG, BRANDON', value='HENG, BRANDON')
df['NAME'] = df['NAME'].replace(regex=r'HENG, FU HONG BRANDON', value='HENG, BRANDON')

df['NAME'] = df['NAME'].replace(regex=r'Tung Hon Andrew Pak', value='PAK, ANDREW')
df['NAME'] = df['NAME'].replace(regex=r'Pak, Andrew', value='PAK, ANDREW')

df['NAME'] = df['NAME'].replace(regex=r'Wong Yijie Lucas', value='WONG YIJIE LUCAS')
df['NAME'] = df['NAME'].replace(regex=r'Wong, Yijie Lucas', value='WONG YIJIE LUCAS')

df['NAME'] = df['NAME'].replace(regex=r'Mun Jern Wei', value='MUN, IVAN')
df['NAME'] = df['NAME'].replace(regex=r'Mun, Jern Wei Ivan', value='MUN, IVAN')
df['NAME'] = df['NAME'].replace(regex=r'Mun, Ivan', value='MUN, IVAN')


df['NAME'] = df['NAME'].replace(regex=r'Conrad Kangli Emery', value='EMERY, CONRAD')
df['NAME'] = df['NAME'].replace(regex=r'Conrad, Kangli Emery', value='EMERY, CONRAD')
df['NAME'] = df['NAME'].replace(regex=r'Kangli Emery Conrad', value='EMERY, CONRAD')
df['NAME'] = df['NAME'].replace(regex=r'Conrad Kangli Emery Q', value='EMERY, CONRAD')

df['NAME'] = df['NAME'].replace(regex=r'Melvin, Wong', value='WONG, MELVIN')
df['NAME'] = df['NAME'].replace(regex=r'MELVIN, WONG', value='WONG, MELVIN')

df['NAME'] = df['NAME'].replace(regex=r'Caleb Hia', value='HIA, CALEB')

df['NAME'] = df['NAME'].replace(regex=r'Jun Yu Low', value='LOW, JUN YU')
df['NAME'] = df['NAME'].replace(regex=r'Low Jun Yu', value='LOW, JUN YU')
df['NAME'] = df['NAME'].replace(regex=r'Low, Jun Yu', value='LOW, JUN YU')

df['NAME'] = df['NAME'].replace(regex=r'Gabriel Lee', value='LEE GABRIEL JIN YI')
df['NAME'] = df['NAME'].replace(regex=r'Lee, Gabriel', value='LEE GABRIEL JIN YI')

df['NAME'] = df['NAME'].replace(regex=r'Andrew George Medina', value='MEDINA, ANDREW GEORGE')
df['NAME'] = df['NAME'].replace(regex=r'Medina, Andrew', value='MEDINA, ANDREW GEORGE')
df['NAME'] = df['NAME'].replace(regex=r'Medina, Andrew George', value='MEDINA, ANDREW GEORGE')
df['NAME'] = df['NAME'].replace(regex=r'George Medina, Andrew', value='MEDINA, ANDREW GEORGE')
df['NAME'] = df['NAME'].replace(regex=r'George George Medina, Andrew', value='MEDINA, ANDREW GEORGE')



df['NAME'] = df['NAME'].replace(regex=r'^Harry\sIrfan\sCurra$', value='HARRY IRFAN CURRAN')
df['NAME'] = df['NAME'].replace(regex=r'Curran, Harry Irfann', value='HARRY IRFAN CURRAN')
df['NAME'] = df['NAME'].replace(regex=r'^Curran,\sHarry$', value='HARRY IRFAN CURRAN')
df['NAME'] = df['NAME'].replace(regex=r'Harry Irfan Curran Q', value='HARRY IRFAN CURRAN')
df['NAME'] = df['NAME'].replace(regex=r'Harry Irfan Curran', value='HARRY IRFAN CURRAN')
df['NAME'] = df['NAME'].replace(regex=r'Curran, Harry Irfan', value='HARRY IRFAN CURRAN')
df['NAME'] = df['NAME'].replace(regex=r'CURRAN, HARRY IRFAN', value='HARRY IRFAN CURRAN')


df['NAME'] = df['NAME'].replace(regex=r'Wei Jun Huang', value='HUANG WEI JUN')
df['NAME'] = df['NAME'].replace(regex=r'HUANG, WEI JUN', value='HUANG WEI JUN')

df['NAME'] = df['NAME'].replace(regex=r'., Hariharan', value=', HARIHARAN')
df['NAME'] = df['NAME'].replace(regex=r'S/O KrishnS/O Krishna, Hariharan', value=', HARIHARAN')
df['NAME'] = df['NAME'].replace(regex=r'KRISHNAN, HARIHARAN S/O', value=', HARIHARAN')

'''

"\n\n# Correct some of the known name variations\n# Need to fix 2024 NSG names extracts (extracting Q/q with name)\n\ndf['NAME'] = df['NAME'].str.replace('\xa0', '', regex=True)\ndf['NAME'] = df['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)\ndf['NAME'] = df['NAME'].str.replace('\r', '', regex=True)\ndf['NAME'] = df['NAME'].str.replace('\n', '', regex=True)\ndf['NAME'] = df['NAME'].str.strip()\n\n\n\ndf['NAME'] = df['NAME'].replace(regex=r'Norhisham, Jamie El Redha Ang El Redha Ang', value='Norhisham, Jamie El Redha Ang')\n\n\n\ndf['NAME'] = df['NAME'].replace(regex=r'TAN, ELIZABETH ANN SHEE R', value='TAN, ELIZABETH-ANN')\ndf['NAME'] = df['NAME'].replace(regex=r'Tan, Elizabeth Ann', value='TAN, ELIZABETH-ANN')\ndf['NAME'] = df['NAME'].replace(regex=r'TAN, SHEE RU, ELIZABETH-A', value='TAN, ELIZABETH-ANN')\ndf['NAME'] = df['NAME'].replace(regex=r'Elizabeth-Ann Tan', value='TAN, ELIZABETH-ANN')\ndf['NAME'] = df['NAME'].replace(regex=r'Tan, Elizabeth-Ann', value='TAN, ELIZAB

In [57]:
'''
# Read a variation name list and corrections from CSVs

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')

names = pd.read_csv("name_variations.csv")

for index, row in names.iterrows():
        
    print(names.VARIATION, names.NAME)
    df['NAME'] = df['NAME'].replace(regex=rf"{row['VARIATION']}", value=f"{row['NAME']}")

'''    

'\n# Read a variation name list and corrections from CSVs\n\nos.chdir(\'/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/\')\n\nnames = pd.read_csv("name_variations.csv")\n\nfor index, row in names.iterrows():\n        \n    print(names.VARIATION, names.NAME)\n    df[\'NAME\'] = df[\'NAME\'].replace(regex=rf"{row[\'VARIATION\']}", value=f"{row[\'NAME\']}")\n\n'

In [58]:
# Read name variations from GCS name lists bucket (Still in beta)

df['NAME'] = df['NAME'].str.replace('\xa0', '', regex=True)
df['NAME'] = df['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\r', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\n', '', regex=True)
df['NAME'] = df['NAME'].str.strip()


# Read csv from GCS bucket

file_path = "gs://name_lists/name_variations.csv"
names = pd.read_csv(file_path,
                 sep=",",
                 storage_options={"token": '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json'})

# Iterate over dataframe and replace names

for index, row in names.iterrows():
        
    df['NAME'] = df['NAME'].replace(regex=rf"{row['VARIATION']}", value=f"{row['NAME']}")


In [59]:
df[df['NAME']=='SOH RUI YONG GUILLAUME']

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,NATIONALITY,...,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
25992,SOH RUI YONG GUILLAUME,None,1:09:28,None,None,38,Half Marathon,None,6-Aug-91,SGP,...,NaN,NaN,4168.0,4168.0,4168.0,NaN,NaN,NaN,NaN,NaN
25993,SOH RUI YONG GUILLAUME,None,15:08,None,None,46,"5km, Road",None,6-Aug-91,SGP,...,NaN,NaN,4168.0,4168.0,4168.0,NaN,NaN,NaN,NaN,NaN
25994,SOH RUI YONG GUILLAUME,None,08:35.8,None,None,12,3000m,None,6-Aug-91,SGP,...,NaN,NaN,515.8,515.8,515.8,NaN,NaN,NaN,NaN,NaN
25995,SOH RUI YONG GUILLAUME,None,32:03:00,None,None,2,"10km, Road",None,6-Aug-91,SGP,...,NaN,NaN,115380.0,115380.0,115380.0,NaN,NaN,NaN,NaN,NaN
25996,SOH RUI YONG GUILLAUME,None,15:06.1,None,None,12,5000m,None,6-Aug-91,SGP,...,914.319,927.57,906.1,906.1,906.1,-5.032,8.219,21.47,-22.7,2.430383
26054,SOH RUI YONG GUILLAUME,None,2:24:07,None,None,41,Marathon,None,6-Aug-91,SGP,...,NaN,NaN,8647.0,8647.0,8647.0,NaN,NaN,NaN,NaN,NaN
26126,SOH RUI YONG GUILLAUME,None,2:27:31,None,None,37,Marathon,None,6-Aug-91,SGP,...,NaN,NaN,8851.0,8851.0,8851.0,NaN,NaN,NaN,NaN,NaN
26206,SOH RUI YONG GUILLAUME,None,1:08:57,None,None,178,Half Marathon,None,6-Aug-91,SGP,...,NaN,NaN,4137.0,4137.0,4137.0,NaN,NaN,NaN,NaN,NaN
26721,SOH RUI YONG GUILLAUME,None,31:11.4,INDIVIDUAL,33,133,10000m,None,1991,SGP,...,NaN,NaN,1871.4,1871.4,1871.4,NaN,NaN,NaN,NaN,NaN
26722,SOH RUI YONG GUILLAUME,None,04:59.3,INDIVIDUAL,33,12,1 mile,None,1991,SGP,...,NaN,NaN,299.3,299.3,299.3,NaN,NaN,NaN,NaN,NaN


In [60]:
# Exclude foreigners from MALAYSIA, THAILAND etc.

df_select = df[(df['TEAM']!='Malaysia') & (df['TEAM']!='THAILAND') & (df['TEAM']!='China') & (df['TEAM']!='South Korea') & (df['TEAM']!='Laos') & (df['TEAM']!='Philippines') & (df['TEAM']!='Piboonbumpen Thailand') & (df['TEAM']!='Chinese Taipei') & (df['TEAM']!='Gurkha Contingent') & (df['TEAM']!='Australia') & (df['TEAM']!='Piboonbumpen Thailand') & (df['TEAM']!='Hong Kong') & (df['TEAM']!='PERAK')] 


In [61]:
df_select

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,NATIONALITY,...,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,"Phuak, Shiyun Hazel",,05:23.4,Club ZOOM,9,10,Run,nan,18/7/15,nan,...,NaN,NaN,323.4,323.4,323.40,NaN,NaN,NaN,NaN,NaN
1,"Sumandran, Esther",,04:18.7,Club ZOOM,9,4,Run,nan,30/8/15,nan,...,NaN,NaN,258.7,258.7,258.70,NaN,NaN,NaN,NaN,NaN
2,"Tei, Avery",,DNS,Club ZOOM,9,0,Run,nan,25/8/15,nan,...,NaN,NaN,258.7,258.7,258.70,NaN,NaN,NaN,NaN,NaN
3,"Tee, Ena",,04:30.7,JS Athletics,9,6,Run,nan,30/3/15,nan,...,NaN,NaN,270.7,270.7,270.70,NaN,NaN,NaN,NaN,NaN
4,"Pistore, Viola",,04:31.3,JS Athletics,9,7,Run,nan,26/8/15,nan,...,NaN,NaN,271.3,271.3,271.30,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28047,FUN LE CONG LUCAS,,15.35,Singapore,18,1,Hurdles,,1/11/06,,...,NaN,NaN,15.35,15.35,15.35,NaN,NaN,NaN,NaN,NaN
28048,"TAN, DARYL HONG AN",,7.16,Singapore,23,1,Dash,,18/12/01,,...,NaN,NaN,7.16,7.16,7.16,NaN,NaN,NaN,NaN,NaN
28049,"HENG, BRANDON",,NH,Singapore,26,0,High Jump,,13/12/98,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28050,"Toh Jun Xi, Tedd",,6.55m,Singapore,0,1,Long Jump,,nan,,...,NaN,NaN,6.55,6.55,6.55,NaN,NaN,NaN,NaN,NaN


In [62]:
df[df['NAME']=='SOH RUI YONG GUILLAUME']

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,NATIONALITY,...,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
25992,SOH RUI YONG GUILLAUME,None,1:09:28,None,None,38,Half Marathon,None,6-Aug-91,SGP,...,NaN,NaN,4168.0,4168.0,4168.0,NaN,NaN,NaN,NaN,NaN
25993,SOH RUI YONG GUILLAUME,None,15:08,None,None,46,"5km, Road",None,6-Aug-91,SGP,...,NaN,NaN,4168.0,4168.0,4168.0,NaN,NaN,NaN,NaN,NaN
25994,SOH RUI YONG GUILLAUME,None,08:35.8,None,None,12,3000m,None,6-Aug-91,SGP,...,NaN,NaN,515.8,515.8,515.8,NaN,NaN,NaN,NaN,NaN
25995,SOH RUI YONG GUILLAUME,None,32:03:00,None,None,2,"10km, Road",None,6-Aug-91,SGP,...,NaN,NaN,115380.0,115380.0,115380.0,NaN,NaN,NaN,NaN,NaN
25996,SOH RUI YONG GUILLAUME,None,15:06.1,None,None,12,5000m,None,6-Aug-91,SGP,...,914.319,927.57,906.1,906.1,906.1,-5.032,8.219,21.47,-22.7,2.430383
26054,SOH RUI YONG GUILLAUME,None,2:24:07,None,None,41,Marathon,None,6-Aug-91,SGP,...,NaN,NaN,8647.0,8647.0,8647.0,NaN,NaN,NaN,NaN,NaN
26126,SOH RUI YONG GUILLAUME,None,2:27:31,None,None,37,Marathon,None,6-Aug-91,SGP,...,NaN,NaN,8851.0,8851.0,8851.0,NaN,NaN,NaN,NaN,NaN
26206,SOH RUI YONG GUILLAUME,None,1:08:57,None,None,178,Half Marathon,None,6-Aug-91,SGP,...,NaN,NaN,4137.0,4137.0,4137.0,NaN,NaN,NaN,NaN,NaN
26721,SOH RUI YONG GUILLAUME,None,31:11.4,INDIVIDUAL,33,133,10000m,None,1991,SGP,...,NaN,NaN,1871.4,1871.4,1871.4,NaN,NaN,NaN,NaN,NaN
26722,SOH RUI YONG GUILLAUME,None,04:59.3,INDIVIDUAL,33,12,1 mile,None,1991,SGP,...,NaN,NaN,299.3,299.3,299.3,NaN,NaN,NaN,NaN,NaN


In [63]:
df_select[(df_select['NAME']=='Tan, Kian Ye') & (df_select['MAPPED_EVENT']=='200m')]

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,NATIONALITY,...,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR


In [64]:
# Remove spurious SEED results 

#df_select=df_select[~((df_select['NAME']=='Tan, Kian Ye') & (df_select['MAPPED_EVENT']=='200m'))]
#df_select=df_select[~((df_select['NAME']=='LOH, BEAVERLY') & (df_select['MAPPED_EVENT']=='400m'))]
#df_select=df_select[~((df_select['NAME']=='Vaz, Luisa Marie') & (df_select['MAPPED_EVENT']=='High jump'))]


In [65]:
df_select

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,NATIONALITY,...,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,"Phuak, Shiyun Hazel",,05:23.4,Club ZOOM,9,10,Run,nan,18/7/15,nan,...,NaN,NaN,323.4,323.4,323.40,NaN,NaN,NaN,NaN,NaN
1,"Sumandran, Esther",,04:18.7,Club ZOOM,9,4,Run,nan,30/8/15,nan,...,NaN,NaN,258.7,258.7,258.70,NaN,NaN,NaN,NaN,NaN
2,"Tei, Avery",,DNS,Club ZOOM,9,0,Run,nan,25/8/15,nan,...,NaN,NaN,258.7,258.7,258.70,NaN,NaN,NaN,NaN,NaN
3,"Tee, Ena",,04:30.7,JS Athletics,9,6,Run,nan,30/3/15,nan,...,NaN,NaN,270.7,270.7,270.70,NaN,NaN,NaN,NaN,NaN
4,"Pistore, Viola",,04:31.3,JS Athletics,9,7,Run,nan,26/8/15,nan,...,NaN,NaN,271.3,271.3,271.30,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28047,FUN LE CONG LUCAS,,15.35,Singapore,18,1,Hurdles,,1/11/06,,...,NaN,NaN,15.35,15.35,15.35,NaN,NaN,NaN,NaN,NaN
28048,"TAN, DARYL HONG AN",,7.16,Singapore,23,1,Dash,,18/12/01,,...,NaN,NaN,7.16,7.16,7.16,NaN,NaN,NaN,NaN,NaN
28049,"HENG, BRANDON",,NH,Singapore,26,0,High Jump,,13/12/98,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28050,"Toh Jun Xi, Tedd",,6.55m,Singapore,0,1,Long Jump,,nan,,...,NaN,NaN,6.55,6.55,6.55,NaN,NaN,NaN,NaN,NaN


In [66]:
'''
# Read list of foreigners

foreigners = pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/List of Foreigners.csv', encoding='latin-1')
'''


"\n# Read list of foreigners\n\nforeigners = pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/List of Foreigners.csv', encoding='latin-1')\n"

In [67]:
# Read list of foreigners from GCS bucket

file_path = "gs://name_lists/List of Foreigners.csv"
foreigners = pd.read_csv(file_path,
                 sep=",",
                 encoding="unicode escape",
                 storage_options={"token": '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json'})


In [68]:
foreigners

,LAST_NAME,FIRST_NAME
0,Aaryan,Greuter Christoph
1,Akahodani,Takayuki
2,Apondar,Audric
3,Brooks,Ruby
4,Brouwer,Cees
...,...,...
236,ISMAIL,MUHAMMAD ZULFIQAR
237,Jayaganeson,Kirtisha
238,LIN,Yu Sian
239,King,Leonard


In [69]:
foreigners['V1'] = foreigners['LAST_NAME']+' '+foreigners['FIRST_NAME']
foreigners['V2'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']
foreigners['V3'] = foreigners['LAST_NAME']+', '+foreigners['FIRST_NAME']
foreigners['V4'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']

for1 = foreigners['V1'].dropna().tolist()
for2 = foreigners['V2'].dropna().tolist()
for3 = foreigners['V3'].dropna().tolist()
for4 = foreigners['V4'].dropna().tolist()

foreign_list = for1+for2+for3+for4 

foreign_list_casefold=[s.casefold() for s in foreign_list]

exclusions = foreign_list_casefold

no_foreigners_list = df_select.loc[~df['NAME'].str.casefold().isin(exclusions)]  # ~ means NOT IN. DROP spex carded athletes

In [70]:
#rslt_df = df.loc[(df['RESULT_CONV'] < df['2pc']) & (df['AGE']<40) & (df['EVENT']!='Marathon')]

In [71]:
#rslt_df = df.loc[(((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))&(df['RESULT_CONV'] >= df['5pc']) & (df['AGE']<40) & ((df['EVENT']!='Marathon')|(df['AGE']<60) & (df['EVENT']=='Marathon')))]

In [72]:
no_foreigners_list

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,NATIONALITY,...,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,"Phuak, Shiyun Hazel",,05:23.4,Club ZOOM,9,10,Run,nan,18/7/15,nan,...,NaN,NaN,323.4,323.4,323.40,NaN,NaN,NaN,NaN,NaN
1,"Sumandran, Esther",,04:18.7,Club ZOOM,9,4,Run,nan,30/8/15,nan,...,NaN,NaN,258.7,258.7,258.70,NaN,NaN,NaN,NaN,NaN
2,"Tei, Avery",,DNS,Club ZOOM,9,0,Run,nan,25/8/15,nan,...,NaN,NaN,258.7,258.7,258.70,NaN,NaN,NaN,NaN,NaN
3,"Tee, Ena",,04:30.7,JS Athletics,9,6,Run,nan,30/3/15,nan,...,NaN,NaN,270.7,270.7,270.70,NaN,NaN,NaN,NaN,NaN
4,"Pistore, Viola",,04:31.3,JS Athletics,9,7,Run,nan,26/8/15,nan,...,NaN,NaN,271.3,271.3,271.30,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28047,FUN LE CONG LUCAS,,15.35,Singapore,18,1,Hurdles,,1/11/06,,...,NaN,NaN,15.35,15.35,15.35,NaN,NaN,NaN,NaN,NaN
28048,"TAN, DARYL HONG AN",,7.16,Singapore,23,1,Dash,,18/12/01,,...,NaN,NaN,7.16,7.16,7.16,NaN,NaN,NaN,NaN,NaN
28049,"HENG, BRANDON",,NH,Singapore,26,0,High Jump,,13/12/98,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28050,"Toh Jun Xi, Tedd",,6.55m,Singapore,0,1,Long Jump,,nan,,...,NaN,NaN,6.55,6.55,6.55,NaN,NaN,NaN,NaN,NaN


In [73]:
# Choose the best performing event for each athlete

#top_performers_clean = excluded_list.sort_values(['NAME','PERF_SCALAR'],ascending=False).groupby('NAME').head(1) # Choose top performing event per NAME


In [74]:
# Choose the best result for each event participated by every athlete

top_performers_clean = no_foreigners_list.sort_values(['MAPPED_EVENT', 'NAME','PERF_SCALAR'],ascending=False).groupby(['MAPPED_EVENT', 'NAME']).head(1)


In [75]:
top_performers_clean.reset_index(inplace=True)


In [76]:
top_performers_clean.to_csv('top_performers_clean_prod.csv', encoding='utf-8')

In [77]:
top_performers_clean

,index,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,...,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,4672,"yeo, husni",,10.09m,Singapore Polytechnic,19,17,Triple Jump,H132F05,1/2/05,...,NaN,NaN,10.09,10.09,10.09,NaN,NaN,NaN,NaN,NaN
1,13786,"ooi, chen si",,9.22m,Ngee Ann Polytechnic,18,9,Triple Jump,nan,30/10/06,...,NaN,NaN,9.22,9.22,9.22,NaN,NaN,NaN,NaN,NaN
2,14163,"kwan, Matthias",,DNS,Unattached,22,0,Triple Jump,nan,26/4/02,...,NaN,NaN,5.94,5.94,5.94,NaN,NaN,NaN,NaN,NaN
3,14106,"kapil, Arnav",,15.03m,WINGS ATHLETICS CLUB,21,1,Triple Jump,nan,25/8/03,...,NaN,NaN,15.03,15.03,15.03,NaN,NaN,NaN,NaN,NaN
4,7340,"chua, keira",,10.16m,Cedar Girls Secondary School,16,5,Triple Jump,nan,18/9/08,...,NaN,NaN,10.16,10.16,10.16,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12814,6576,"., Amri Muqtadar",,DNS,NJAC (MAS),20,0,Dash,nan,6/9/04,...,NaN,NaN,12.4,12.4,12.40,NaN,NaN,NaN,NaN,NaN
12815,5694,"., Aliya",,DNS,NJAC (MAS),10,0,Run,nan,30/3/14,...,NaN,NaN,121.8,121.8,121.80,NaN,NaN,NaN,NaN,NaN
12816,6349,"., Abby Ho",,15.56,North Vista,14,10,Dash,nan,1/1/10,...,NaN,NaN,15.56,15.56,15.56,NaN,NaN,NaN,NaN,NaN
12817,3529,"., AaLiyah",,02:50.4,Erovra Club,15,2,Run,nan,10/12/08,...,NaN,NaN,170.4,170.4,170.40,NaN,NaN,NaN,NaN,NaN


In [78]:
# Join 2024 best results for each event for each athlete

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/2023/')


df_yoy = pd.read_csv("Best_results_2023.csv")

df_yoy['NAME'] = df_yoy['NAME'].str.replace('\xa0', '', regex=True)
df_yoy['NAME'] = df_yoy['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df_yoy['NAME'] = df_yoy['NAME'].str.replace('\r', '', regex=True)
df_yoy['NAME'] = df_yoy['NAME'].str.replace('\n', '', regex=True)
df_yoy['NAME'] = df_yoy['NAME'].str.strip()


In [79]:
df_yoy

,EVENT,NAME,GENDER,TEAM,BEST_RESULT(2023)
0,100m,"CHUA, JOSHUA",Male,SINGAPORE,10.59
1,100m,"RAPHAEL, RYAN",Male,SINGAPORE,10.72
2,100m,"HO ANN HENG, XANDER",Male,NANYANG TECHNOLOGICAL UNI,10.73
3,100m,TATE TAN FUNG,Male,RI,10.74
4,100m,"TAN, DARYL",Male,SINGAPORE INSTITUTE OF TECHNOL,10.91
5,100m,"ONG, YING TAT",Male,VICTORIA JUNIOR COLLEGE,11.1
6,100m,"RYAN, PRAHARSH",Male,NANYANG TECHNOLOGICAL UNI,10.99
7,10000m,"LIM, JON",Male,NATIONAL UNIVERSITY SINGAPORE,NaN
8,110m hurdles,"CHONG, WEI GUAN",Male,NATIONAL UNIVERSITY OF SINGAPO,14.29
9,1500m,"SOUNDARARAJAH, JEEVANEESH",Male,INDIVIDUAL,04:06.5


In [80]:
# Merge benchmarks onto athletes on MAPPED_EVENT and GENDER

yoy_performance = pd.merge(
    left=df_yoy, 
    right=top_performers_clean,
    how='left',
    left_on=['EVENT', 'GENDER', 'NAME'],
    right_on=['MAPPED_EVENT', 'GENDER', 'NAME'],
)

In [81]:
yoy_performance

,EVENT,NAME,GENDER,TEAM_x,BEST_RESULT(2023),index,SEED,RESULT,TEAM_y,AGE,...,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,100m,"CHUA, JOSHUA",Male,SINGAPORE,10.59,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100m,"RAPHAEL, RYAN",Male,SINGAPORE,10.72,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100m,"HO ANN HENG, XANDER",Male,NANYANG TECHNOLOGICAL UNI,10.73,26467.0,None,10.91,None,None,...,11.55060,11.7180,10.91,10.91,10.91,0.4732,0.64060,0.8080,0.25,7.240143
3,100m,TATE TAN FUNG,Male,RI,10.74,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100m,"TAN, DARYL",Male,SINGAPORE INSTITUTE OF TECHNOL,10.91,26086.0,None,10.68,None,None,...,11.55060,11.7180,10.68,10.68,10.68,0.7032,0.87060,1.0380,0.48,9.301075
5,100m,"ONG, YING TAT",Male,VICTORIA JUNIOR COLLEGE,11.1,26605.0,None,10.68,None,None,...,11.55060,11.7180,10.68,10.68,10.68,0.7032,0.87060,1.0380,0.48,9.301075
6,100m,"RYAN, PRAHARSH",Male,NANYANG TECHNOLOGICAL UNI,10.99,26451.0,None,11.19,None,None,...,11.55060,11.7180,11.19,11.19,11.19,0.1932,0.36060,0.5280,-0.03,4.731183
7,10000m,"LIM, JON",Male,NATIONAL UNIVERSITY SINGAPORE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,110m hurdles,"CHONG, WEI GUAN",Male,NATIONAL UNIVERSITY OF SINGAPO,14.29,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1500m,"SOUNDARARAJAH, JEEVANEESH",Male,INDIVIDUAL,04:06.5,26731.0,None,4:13:02,INDIVIDUAL,31,...,247.77900,251.3700,15182.0,15182.0,15182.00,14937.8120,14934.22100,14930.6300,14942.60,6236.687552


In [82]:
yoy_performance.to_csv('yoy_performance_prod.csv', index=False, encoding='utf-8')

In [83]:
tiered_performers = top_performers_clean.sort_values(['GENDER', 'MAPPED_EVENT', 'PERF_SCALAR'],ascending=False).groupby(['MAPPED_EVENT', 'NAME']).head(1)


In [84]:
tiered_performers

,index,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,...,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
7073,27974,"{'884.0': 'Ahamed, Sahnazbegum Basheer', '749....",,52.82,Team Fabian,nan,1,Relay,,nan,...,NaN,NaN,52.82,52.82,52.82,NaN,NaN,NaN,NaN,NaN
7077,27968,"{'880.0': 'HEBER RUIZ , ARRATIA', '1716.0': '....",,01:11.4,Energetix Pte Ltd,nan,7,Relay,,nan,...,NaN,NaN,71.4,71.4,71.40,NaN,NaN,NaN,NaN,NaN
7082,27976,"{'871.0': 'Heng, Yu Xuan Sarah'}",,DNS,Erovra Club,nan,0,Relay,,nan,...,NaN,NaN,23.41,23.41,23.41,NaN,NaN,NaN,NaN,NaN
7100,27959,"{'845.0': 'Koh, Kok Siong', '1455.0': 'KOH, Dy...",,15.47,Unattached,nan,4,Relay,,nan,...,NaN,NaN,15.47,15.47,15.47,NaN,NaN,NaN,NaN,NaN
7101,28023,"{'845.0': 'Koh, Kok Siong', '1444.0': 'KOH, De...",,15.97,Unattached,nan,7,Relay,,nan,...,NaN,NaN,15.97,15.97,15.97,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12805,9152,"., Dharkshitha",,12:46.5,Cedar Girls Secondary School,14,5,Race Walk,nan,6/4/10,...,NaN,NaN,766.5,766.5,766.50,NaN,NaN,NaN,NaN,NaN
12815,5694,"., Aliya",,DNS,NJAC (MAS),10,0,Run,nan,30/3/14,...,NaN,NaN,121.8,121.8,121.80,NaN,NaN,NaN,NaN,NaN
12816,6349,"., Abby Ho",,15.56,North Vista,14,10,Dash,nan,1/1/10,...,NaN,NaN,15.56,15.56,15.56,NaN,NaN,NaN,NaN,NaN
12817,3529,"., AaLiyah",,02:50.4,Erovra Club,15,2,Run,nan,10/12/08,...,NaN,NaN,170.4,170.4,170.40,NaN,NaN,NaN,NaN,NaN


In [85]:
# Filter top 8 performances for each event and gender

#top_performers = top_performers_clean.sort_values(by=['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'],ascending=[False, False, False]).groupby(['GENDER', 'MAPPED_EVENT']).head(8).reset_index(drop=True)




In [86]:
# Convert time format for marathon and 5000m into mm:ss.00
import datetime

#s=247.779

#datetime.datetime.fromtimestamp(s).strftime('%M:%S.%f')

for i in range(len(top_performers_clean)):
        
    rowIndex = top_performers_clean.index[i]

    event=top_performers_clean.iloc[rowIndex,17]
        
    
    time_base2=top_performers_clean.iloc[rowIndex,21]
    time_base3=top_performers_clean.iloc[rowIndex,22]
    time_base5=top_performers_clean.iloc[rowIndex,23]
    
        
    if metric==None:
        continue
        
    if event=='800m' or event=='10000m' or event=='5000m' or event=='3000m steeplechase' or event=='1500m':
        
        print(time_base2, time_base3, time_base5)

        
        
        try:
            
        
            date_preconvert2 = datetime.datetime.utcfromtimestamp(time_base2)
            date_preconvert3 = datetime.datetime.utcfromtimestamp(time_base3)
            date_preconvert5 = datetime.datetime.utcfromtimestamp(time_base5)

            output2 = datetime.datetime.strftime(date_preconvert2, "%M:%S.%f")
            output3 = datetime.datetime.strftime(date_preconvert3, "%M:%S.%f")
            output5 = datetime.datetime.strftime(date_preconvert5, "%M:%S.%f")
            
            print(output2, output3, output5)

                    
            top_performers_clean.loc[rowIndex, '2%_timing'] = output2
            top_performers_clean.loc[rowIndex, '3.5%_timing'] = output3
            top_performers_clean.loc[rowIndex, '5%_timing'] = output5
            

            
        except:
            
            pass
        

        
    elif event=='Marathon':
        
        print(time_base2, time_base3, time_base5)

        
        try:
            

        
            date_preconvert2 = datetime.datetime.utcfromtimestamp(time_base2)
            date_preconvert3 = datetime.datetime.utcfromtimestamp(time_base3)
            date_preconvert5 = datetime.datetime.utcfromtimestamp(time_base5)

            
            
            output2 = datetime.datetime.strftime(date_preconvert2, "%H:%M:%S")
            output3 = datetime.datetime.strftime(date_preconvert3, "%H:%M:%S")
            output5 = datetime.datetime.strftime(date_preconvert5, "%H:%M:%S")

            
        
            top_performers_clean.loc[rowIndex, '2%_timing'] = output2
            top_performers_clean.loc[rowIndex, '3.5%_timing'] = output3
            top_performers_clean.loc[rowIndex, '5%_timing'] = output5
            
            print('output', output2, output3, output5)


        
        except:
            
            pass
                        
             


nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
117.19800000000001 118.9215 120.64500000000001
01:57.198000 01:58.921500 02:00.645000
152.898 155.1465 157.395
02:32.898000 02:35.146500 02:37.395000
117.19800000000001 118.9215 120.64500000000001
01:57.198000 01:58.921500 02:00.645000
117.19800000000001 118.9215 120.64500000000001
01:57.198000 01:58.921500 02:00.645000
117.19800000000001 118.9215 120.64500000000001
01:57.198000 01:58.921500 02:00.645000
117.19800000000001 118.9215 120.64500000000001
01:57.198000 01:58.921500 02:00.645000
152.898 155.1465 157.395
0

117.19800000000001 118.9215 120.64500000000001
01:57.198000 01:58.921500 02:00.645000
117.19800000000001 118.9215 120.64500000000001
01:57.198000 01:58.921500 02:00.645000
117.19800000000001 118.9215 120.64500000000001
01:57.198000 01:58.921500 02:00.645000
117.19800000000001 118.9215 120.64500000000001
01:57.198000 01:58.921500 02:00.645000
117.19800000000001 118.9215 120.64500000000001
01:57.198000 01:58.921500 02:00.645000
152.898 155.1465 157.395
02:32.898000 02:35.146500 02:37.395000
152.898 155.1465 157.395
02:32.898000 02:35.146500 02:37.395000
152.898 155.1465 157.395
02:32.898000 02:35.146500 02:37.395000
152.898 155.1465 157.395
02:32.898000 02:35.146500 02:37.395000
152.898 155.1465 157.395
02:32.898000 02:35.146500 02:37.395000
117.19800000000001 118.9215 120.64500000000001
01:57.198000 01:58.921500 02:00.645000
152.898 155.1465 157.395
02:32.898000 02:35.146500 02:37.395000
152.898 155.1465 157.395
02:32.898000 02:35.146500 02:37.395000
117.19800000000001 118.9215 120.6450

358.836 364.113 369.39000000000004
05:58.836000 06:04.113000 06:09.390000
271.62600000000003 275.6205 279.615
04:31.626000 04:35.620500 04:39.615000
358.836 364.113 369.39000000000004
05:58.836000 06:04.113000 06:09.390000
358.836 364.113 369.39000000000004
05:58.836000 06:04.113000 06:09.390000
358.836 364.113 369.39000000000004
05:58.836000 06:04.113000 06:09.390000
358.836 364.113 369.39000000000004
05:58.836000 06:04.113000 06:09.390000
358.836 364.113 369.39000000000004
05:58.836000 06:04.113000 06:09.390000
358.836 364.113 369.39000000000004
05:58.836000 06:04.113000 06:09.390000
244.18800000000002 247.779 251.37
04:04.188000 04:07.779000 04:11.370000
358.836 364.113 369.39000000000004
05:58.836000 06:04.113000 06:09.390000
271.62600000000003 275.6205 279.615
04:31.626000 04:35.620500 04:39.615000
271.62600000000003 275.6205 279.615
04:31.626000 04:35.620500 04:39.615000
271.62600000000003 275.6205 279.615
04:31.626000 04:35.620500 04:39.615000
358.836 364.113 369.39000000000004


In [87]:
top_performers_clean

,index,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,...,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,2%_timing,3.5%_timing,5%_timing
0,4672,"yeo, husni",,10.09m,Singapore Polytechnic,19,17,Triple Jump,H132F05,1/2/05,...,10.09,10.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13786,"ooi, chen si",,9.22m,Ngee Ann Polytechnic,18,9,Triple Jump,nan,30/10/06,...,9.22,9.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14163,"kwan, Matthias",,DNS,Unattached,22,0,Triple Jump,nan,26/4/02,...,5.94,5.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14106,"kapil, Arnav",,15.03m,WINGS ATHLETICS CLUB,21,1,Triple Jump,nan,25/8/03,...,15.03,15.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7340,"chua, keira",,10.16m,Cedar Girls Secondary School,16,5,Triple Jump,nan,18/9/08,...,10.16,10.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12814,6576,"., Amri Muqtadar",,DNS,NJAC (MAS),20,0,Dash,nan,6/9/04,...,12.4,12.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12815,5694,"., Aliya",,DNS,NJAC (MAS),10,0,Run,nan,30/3/14,...,121.8,121.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12816,6349,"., Abby Ho",,15.56,North Vista,14,10,Dash,nan,1/1/10,...,15.56,15.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12817,3529,"., AaLiyah",,02:50.4,Erovra Club,15,2,Run,nan,10/12/08,...,170.4,170.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
# Identify Tier 1/2/3 performers

top_performers_clean['TIER'] = np.where((top_performers_clean['Delta_Benchmark']>=0), 'Tier 1',    
                                np.where(((top_performers_clean['Delta_Benchmark']<0) & (top_performers_clean['Delta2']>=0)), 'Tier2',
                                np.where(((top_performers_clean['Delta2']<0) & (top_performers_clean['Delta3.5']>=0)), 'Tier3', ' ')))



In [89]:
top_performers_clean

,index,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,...,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,2%_timing,3.5%_timing,5%_timing,TIER
0,4672,"yeo, husni",,10.09m,Singapore Polytechnic,19,17,Triple Jump,H132F05,1/2/05,...,10.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,13786,"ooi, chen si",,9.22m,Ngee Ann Polytechnic,18,9,Triple Jump,nan,30/10/06,...,9.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,14163,"kwan, Matthias",,DNS,Unattached,22,0,Triple Jump,nan,26/4/02,...,5.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,14106,"kapil, Arnav",,15.03m,WINGS ATHLETICS CLUB,21,1,Triple Jump,nan,25/8/03,...,15.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,7340,"chua, keira",,10.16m,Cedar Girls Secondary School,16,5,Triple Jump,nan,18/9/08,...,10.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12814,6576,"., Amri Muqtadar",,DNS,NJAC (MAS),20,0,Dash,nan,6/9/04,...,12.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
12815,5694,"., Aliya",,DNS,NJAC (MAS),10,0,Run,nan,30/3/14,...,121.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
12816,6349,"., Abby Ho",,15.56,North Vista,14,10,Dash,nan,1/1/10,...,15.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
12817,3529,"., AaLiyah",,02:50.4,Erovra Club,15,2,Run,nan,10/12/08,...,170.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [90]:
# Drop rows without a SEAG benchmark

final_df = top_performers_clean[top_performers_clean['BENCHMARK'].notna()]


In [91]:
final_df.to_csv('top_performers_tiered_prod.csv', encoding='utf-8')

In [2130]:
# Apply Rule 4 above...if an athlete is in more than one event, choose the best performing one


#top_performers_final = top_performers_sorted.sort_values(by=['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'], ascending=[False, False, False], na_position='last')


In [3934]:
# Apply Rule 3 & is within 5% performance band

octc_df = df_top.loc[(((df_top['CATEGORY_EVENT']=='Mid')|(df_top['CATEGORY_EVENT']=='Sprint')|(df_top['CATEGORY_EVENT']=='Long')|(df['CATEGORY_EVENT']=='Hurdles')|(df['CATEGORY_EVENT']=='Walk')|(df['CATEGORY_EVENT']=='Relay')|(df['CATEGORY_EVENT']=='Marathon')|(df_top['CATEGORY_EVENT']=='Steeple')|(df_top['CATEGORY_EVENT']=='Pentathlon')|(df_top['CATEGORY_EVENT']=='Heptathlon')|(df_top['CATEGORY_EVENT']=='Triathlon'))&(df_top['RESULT_CONV'] <= df_top['5%']) & (df_top['AGE']<40) & ((df_top['MAPPED_EVENT']!='Marathon')|(df_top['AGE']<60) & (df_top['MAPPED_EVENT']=='Marathon')))|(((df_top['CATEGORY_EVENT']=='Jump')|(df_top['CATEGORY_EVENT']=='Throw'))&(df_top['RESULT_CONV'] >= df_top['5%']) & (df_top['AGE']<40) & ((df_top['MAPPED_EVENT']!='Marathon')|(df_top['AGE']<60) & (df_top['MAPPED_EVENT']=='Marathon')))]

In [3935]:
octc_df

,NAME,RESULT,AGE,COMPETITION_RANK,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,...,EVENT_y,Metric,2%,3.5%,5%,RESULT_CONV,Delta2,Delta3.5,Delta5,PERF_SCALAR
0,"Ting Li King, Brandon",15.03m,21.0,1,Men Triple Jump Open,2002-01-09 00:00:00.000,None,Jump,Male,83rd Singapore Open Track & Field,...,Triple jump,15.70,15.386,15.15050,14.9150,15.03,-0.356,-0.12050,0.1150,0.732484
1,"ANURA, ANDRE",14.93m,24.0,2,Men Triple Jump Open,1999-06-12 00:00:00.000,None,Jump,Male,83rd Singapore Open Track & Field,...,Triple jump,15.70,15.386,15.15050,14.9150,14.93,-0.456,-0.22050,0.0150,0.095541
16,"CHEN, Yi Chu",16.87m,21.0,1,Men Shot Put Open,2002-10-31 00:00:00.000,None,Throw,Male,83rd Singapore Open Track & Field,...,Shot put,17.30,16.954,16.69450,16.4350,16.87,-0.084,0.17550,0.4350,2.514451
17,"RIGAN, JONAH CHANG",16.50m,20.0,2,Men Shot Put Open,2003-03-25 00:00:00.000,None,Throw,Male,83rd Singapore Open Track & Field,...,Shot put,17.30,16.954,16.69450,16.4350,16.50,-0.454,-0.19450,0.0650,0.375723
20,"Loh, Anson",15.71m,15.0,1,Men Shot Put U18,2008-04-28,None,Throw,Male,52nd SA Inter Club Championships 2023,...,Shot put,17.30,16.954,16.69450,16.4350,15.71,-1.244,-0.98450,-0.7250,-4.190751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,"PEREIRA, VERONICA SHANTI",11.43,27.0,1,Women 100 Meter Dash Open,1996-09-20 00:00:00.000,None,Sprint,Female,83rd Singapore Open Track & Field,...,100m,11.75,11.985,12.16125,12.3375,11.43,0.555,0.73125,0.9075,7.723404
537,"HU, Chia-Chen",11.57,26.0,2,Women 100 Meter Dash Open,1997-03-28 00:00:00.000,None,Sprint,Female,83rd Singapore Open Track & Field,...,100m,11.75,11.985,12.16125,12.3375,11.57,0.415,0.59125,0.7675,6.531915
540,"TAN, ELIZABETH-ANN",11.91,20.0,3,Women 100 Meter Dash Open,2003-09-23 00:00:00.000,None,Sprint,Female,83rd Singapore Open Track & Field,...,100m,11.75,11.985,12.16125,12.3375,11.91,0.075,0.25125,0.4275,3.638298
555,"Gurung, Subas",31:44.38,26.0,1,Men 10000 Meter Run Open,1997-10-22 00:00:00.000,None,Long,Male,SA Allcomers Meet 2,...,10000m,1885.55,1923.261,1951.54425,1979.8275,1904.38,18.881,47.16425,75.4475,4.001352


In [3936]:
octc_df

,NAME,RESULT,AGE,COMPETITION_RANK,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,...,EVENT_y,Metric,2%,3.5%,5%,RESULT_CONV,Delta2,Delta3.5,Delta5,PERF_SCALAR
0,"Ting Li King, Brandon",15.03m,21.0,1,Men Triple Jump Open,2002-01-09 00:00:00.000,None,Jump,Male,83rd Singapore Open Track & Field,...,Triple jump,15.70,15.386,15.15050,14.9150,15.03,-0.356,-0.12050,0.1150,0.732484
1,"ANURA, ANDRE",14.93m,24.0,2,Men Triple Jump Open,1999-06-12 00:00:00.000,None,Jump,Male,83rd Singapore Open Track & Field,...,Triple jump,15.70,15.386,15.15050,14.9150,14.93,-0.456,-0.22050,0.0150,0.095541
16,"CHEN, Yi Chu",16.87m,21.0,1,Men Shot Put Open,2002-10-31 00:00:00.000,None,Throw,Male,83rd Singapore Open Track & Field,...,Shot put,17.30,16.954,16.69450,16.4350,16.87,-0.084,0.17550,0.4350,2.514451
17,"RIGAN, JONAH CHANG",16.50m,20.0,2,Men Shot Put Open,2003-03-25 00:00:00.000,None,Throw,Male,83rd Singapore Open Track & Field,...,Shot put,17.30,16.954,16.69450,16.4350,16.50,-0.454,-0.19450,0.0650,0.375723
20,"Loh, Anson",15.71m,15.0,1,Men Shot Put U18,2008-04-28,None,Throw,Male,52nd SA Inter Club Championships 2023,...,Shot put,17.30,16.954,16.69450,16.4350,15.71,-1.244,-0.98450,-0.7250,-4.190751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,"PEREIRA, VERONICA SHANTI",11.43,27.0,1,Women 100 Meter Dash Open,1996-09-20 00:00:00.000,None,Sprint,Female,83rd Singapore Open Track & Field,...,100m,11.75,11.985,12.16125,12.3375,11.43,0.555,0.73125,0.9075,7.723404
537,"HU, Chia-Chen",11.57,26.0,2,Women 100 Meter Dash Open,1997-03-28 00:00:00.000,None,Sprint,Female,83rd Singapore Open Track & Field,...,100m,11.75,11.985,12.16125,12.3375,11.57,0.415,0.59125,0.7675,6.531915
540,"TAN, ELIZABETH-ANN",11.91,20.0,3,Women 100 Meter Dash Open,2003-09-23 00:00:00.000,None,Sprint,Female,83rd Singapore Open Track & Field,...,100m,11.75,11.985,12.16125,12.3375,11.91,0.075,0.25125,0.4275,3.638298
555,"Gurung, Subas",31:44.38,26.0,1,Men 10000 Meter Run Open,1997-10-22 00:00:00.000,None,Long,Male,SA Allcomers Meet 2,...,10000m,1885.55,1923.261,1951.54425,1979.8275,1904.38,18.881,47.16425,75.4475,4.001352


In [1022]:
octc_df.to_csv('octc_df.csv', sep=',', encoding='utf-8-sig', index=False)


In [364]:
#rslt_df['Rank'] = (rslt_df.sort_values(by=['EVENT', 'GENDER', 'Delta35'], ascending=[False, False, True])['Delta35']
#                .rank(method='first', ascending=False)
#             )


# Download list of foreigners and spex carded athletes

In [365]:
#spex_list=pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/SPEX_CARDED_LIST.csv', encoding='latin-1')


In [431]:
foreigners = pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/List of Foreigners.csv', encoding='latin-1')


In [367]:
#spex_list

,V1,V2,V3,V4,V5,Level
0,Goh Chui Ling,"Goh, Chui Ling",NaN,NaN,NaN,E3
1,Michelle Sng,"Sng, Michelle",NaN,NaN,NaN,E3
2,Quek Jun Jie Calvin,"Quek, Jun Jie Calvin","QUEK, CALVIN JUN JIE",NaN,NaN,E3
3,"SOH RUI YONG, GUILLAUME","Soh Rui Yong, Guillaume",NaN,NaN,NaN,E3
4,Aaron Justin tan wen jie,"Tan Wen Jie, Aaron Justin",NaN,NaN,NaN,E3P
5,DANIEL LEOW SOON YEE,"Yee, Daniel Leow Soon",NaN,NaN,NaN,E3P
6,Joshua Chua,"Chua, Joshua",NaN,NaN,NaN,E3P
7,NG ZHI RONG RYAN RAPHAEL,"Ng Zhi Rong, Ryan Raphael","Raphael, Ryan",NaN,NaN,E3P
8,Wenli Rachel,"Wenli, Rachel",NaN,NaN,NaN,E3P
9,Wong YaoHan Melvin,"Wong YaoHan, Melvin",NaN,NaN,NaN,E3P


In [432]:
foreigners

,LAST_NAME,FIRST_NAME,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Aaryan,Greuter Christoph,NaN,NaN,NaN
1,Akahodani,Takayuki,NaN,NaN,NaN
2,Apondar,Audric,NaN,NaN,NaN
3,Brooks,Ruby,NaN,NaN,NaN
4,Brouwer,Cees,NaN,NaN,NaN
...,...,...,...,...,...
219,CHO,CHIA-HSUAN,NaN,NaN,NaN
220,NGUYEN,HOAI VAN,NaN,NaN,NaN
221,NGUYEN,QUOC THINH,NaN,NaN,NaN
222,PHAM,QUYNH GIANG,NaN,NaN,NaN


In [433]:
foreigners['V1'] = foreigners['LAST_NAME']+' '+foreigners['FIRST_NAME']
foreigners['V2'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']
foreigners['V3'] = foreigners['LAST_NAME']+', '+foreigners['FIRST_NAME']
foreigners['V4'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']

In [434]:
foreigners

,LAST_NAME,FIRST_NAME,Unnamed: 2,Unnamed: 3,Unnamed: 4,V1,V2,V3,V4
0,Aaryan,Greuter Christoph,NaN,NaN,NaN,Aaryan Greuter Christoph,Greuter Christoph Aaryan,"Aaryan, Greuter Christoph",Greuter Christoph Aaryan
1,Akahodani,Takayuki,NaN,NaN,NaN,Akahodani Takayuki,Takayuki Akahodani,"Akahodani, Takayuki",Takayuki Akahodani
2,Apondar,Audric,NaN,NaN,NaN,Apondar Audric,Audric Apondar,"Apondar, Audric",Audric Apondar
3,Brooks,Ruby,NaN,NaN,NaN,Brooks Ruby,Ruby Brooks,"Brooks , Ruby",Ruby Brooks
4,Brouwer,Cees,NaN,NaN,NaN,Brouwer Cees,Cees Brouwer,"Brouwer, Cees",Cees Brouwer
...,...,...,...,...,...,...,...,...,...
219,CHO,CHIA-HSUAN,NaN,NaN,NaN,CHO CHIA-HSUAN,CHIA-HSUAN CHO,"CHO, CHIA-HSUAN",CHIA-HSUAN CHO
220,NGUYEN,HOAI VAN,NaN,NaN,NaN,NGUYEN HOAI VAN,HOAI VAN NGUYEN,"NGUYEN , HOAI VAN",HOAI VAN NGUYEN
221,NGUYEN,QUOC THINH,NaN,NaN,NaN,NGUYEN QUOC THINH,QUOC THINH NGUYEN,"NGUYEN , QUOC THINH",QUOC THINH NGUYEN
222,PHAM,QUYNH GIANG,NaN,NaN,NaN,PHAM QUYNH GIANG,QUYNH GIANG PHAM,"PHAM , QUYNH GIANG",QUYNH GIANG PHAM


In [371]:
#spex1 = spex_list['V1'].dropna().tolist()
#spex2 = spex_list['V2'].dropna().tolist()
#spex3 = spex_list['V3'].dropna().tolist()
#spex4 = spex_list['V4'].dropna().tolist()
#spex5 = spex_list['V5'].dropna().tolist()


In [435]:
for1 = foreigners['V1'].dropna().tolist()
for2 = foreigners['V2'].dropna().tolist()
for3 = foreigners['V3'].dropna().tolist()
for4 = foreigners['V4'].dropna().tolist()


In [373]:
#spex_athletes=spex1+spex2+spex3+spex4+spex5

In [436]:
foreign_list = for1+for2+for3+for4 

In [437]:
#spex_athletes

In [438]:
foreign_list

['Aaryan Greuter Christoph',
 'Akahodani Takayuki',
 'Apondar Audric',
 'Brooks  Ruby',
 'Brouwer Cees',
 'CARLESATER ARMAAN',
 'Chary Mathangi ',
 'Cinthikael Angel',
 'Comia Tsang Hannah',
 'Comia Tsang Mhandy',
 'Donnelly Jordan ',
 'Dorai Trinity Sharilyn',
 'Freeman Alexander',
 'Gaume Constance',
 'Gregorio John Alexander',
 'HAGIWARA RYOTA ',
 'Indarto Gerard',
 'Joy Matthew',
 'JULURI ADITYA',
 'Kiet Tran',
 'Koduru Kalyani',
 'Kuchenbuch Natalya',
 'KUIJPERS KUIJPERS AYKO ELLIN',
 'Kumar Joseph Suraj',
 'KUPPUSAMY PRAKSHEETA',
 'Kusumo Naia',
 'Kuwalekar  Spruha ',
 'Lahey Kezia Stephanie ',
 'Liz Jerry',
 'Marican Raees Marican Bin Ayoob Marican',
 'MARIE BRUNO DE FERRIERES DE SAUVEBOEUF LOUIS ',
 'Maslov Timofei',
 'Masrezwan Maeva',
 'MENDOZA NICCOLO',
 'Mennella Leonardo',
 'Meyers Nikita Mae Jiny-Yu',
 'Mishra Sunay',
 'Mo Thant Cin',
 'Moon Daniel Youngjo',
 'OCAMPO ANIAG DWAYNE MICHAEL OCAMPO ANIAG',
 'Ortega Rafael Pedro',
 'Owen Elia',
 'owl emma',
 'Ozbudak Deniz',
 

In [439]:
#spex_athletes_casefold=[s.casefold() for s in spex_athletes]

In [440]:
foreign_list_casefold=[s.casefold() for s in foreign_list]

In [442]:
#spex_athletes_casefold

In [443]:
foreign_list_casefold

['aaryan greuter christoph',
 'akahodani takayuki',
 'apondar audric',
 'brooks  ruby',
 'brouwer cees',
 'carlesater armaan',
 'chary mathangi ',
 'cinthikael angel',
 'comia tsang hannah',
 'comia tsang mhandy',
 'donnelly jordan ',
 'dorai trinity sharilyn',
 'freeman alexander',
 'gaume constance',
 'gregorio john alexander',
 'hagiwara ryota ',
 'indarto gerard',
 'joy matthew',
 'juluri aditya',
 'kiet tran',
 'koduru kalyani',
 'kuchenbuch natalya',
 'kuijpers kuijpers ayko ellin',
 'kumar joseph suraj',
 'kuppusamy praksheeta',
 'kusumo naia',
 'kuwalekar  spruha ',
 'lahey kezia stephanie ',
 'liz jerry',
 'marican raees marican bin ayoob marican',
 'marie bruno de ferrieres de sauveboeuf louis ',
 'maslov timofei',
 'masrezwan maeva',
 'mendoza niccolo',
 'mennella leonardo',
 'meyers nikita mae jiny-yu',
 'mishra sunay',
 'mo thant cin',
 'moon daniel youngjo',
 'ocampo aniag dwayne michael ocampo aniag',
 'ortega rafael pedro',
 'owen elia',
 'owl emma',
 'ozbudak deniz',
 

In [444]:
# Exclusion list is foreigners + spex carded 

#exclusions = foreign_list_casefold + spex_athletes_casefold

exclusions = foreign_list_casefold

In [445]:
# Apply Rule 4 above...if an athlete is in more than one event, choose the best performing one

top_performers=rslt_df.sort_values(['NAME','PERF_SCALAR'],ascending=False).groupby('NAME').head(1) # Choose top performing event per NAME


In [446]:
top_performers

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,EVENT_y,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5,PERF_SCALAR
8353,11670,{},12.99,8.0,2,Boys 4x100 Meter Relay 15-16,2015-01-01 00:00:00.000,None,Relay,Male,...,4 x 100m relay,39.36,40.1472,40.73760,41.3280,12.99,27.1572,27.74760,28.3380,71.996951
12480,17949,"{9: 'Ng, Caitlin Shan Wen', 438: 'Seow, Kyra',...",22.17,0.0,1,Girls 13-14 4x100 Meter Relay U15,None,None,Relay,Female,...,4 x 100m relay,44.58,45.4716,46.14030,46.8090,22.17,23.3016,23.97030,24.6390,55.269179
9267,13141,"{995: 'Lee, Asher', 991: 'Yeo, Chee Hean Phili...",29.89,0.0,2,Boys 9-12 4x400 Meter Relay U13,None,None,Relay,Male,...,4 x 400m relay,188.82,192.5964,195.42870,198.2610,29.89,162.7064,165.53870,168.3710,89.170109
9588,13486,"{994: 'De Ming, Gilbert Lee', 988: 'Lee, Jerem...",38.89,0.0,14,Boys 10-12 4x100 Meter Relay U13,None,None,Relay,Male,...,4 x 100m relay,39.36,40.1472,40.73760,41.3280,38.89,1.2572,1.84760,2.4380,6.194106
12424,17876,"{991: 'Yeo, Chee Hean Philip', 992: 'Yeo, Chee...",12.92,0.0,16,Boys 10-12 4x100 Meter Relay U13,None,None,Relay,Male,...,4 x 100m relay,39.36,40.1472,40.73760,41.3280,12.92,27.2272,27.81760,28.4080,72.174797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10975,15886,"., Khairulnazim",14.72,17.0,4,Men 400 Meter Dash Open,2006-01-27,None,Sprint,Male,...,400m,46.63,47.5626,48.26205,48.9615,14.72,32.8426,33.54205,34.2415,73.432340
9918,13873,"., Kaarthika",12.53,27.0,9,Women 400 Meter Dash Women Junior,1996-06-13,None,Sprint,Female,...,400m,53.84,54.9168,55.72440,56.5320,12.53,42.3868,43.19440,44.0020,81.727340
10701,15018,"., Jayashree",1:14.05,15.0,36,Girls Long Jump B Div,2008-01-29 00:00:00.000,None,Jump,Female,...,Long jump,6.02,5.8996,5.80930,5.7190,10.97,5.0704,5.16070,5.2510,87.225914
12077,17484,"., Cheung Zheng",47.61,14.0,20,Boys 11-14 400 Meter Dash U15,2009-02-19,None,Sprint,Male,...,400m,46.63,47.5626,48.26205,48.9615,47.61,-0.0474,0.65205,1.3515,2.898349


In [447]:
top_performers.to_csv('top_checkpoint.csv', sep=',', encoding='utf-8-sig', index=False)

In [448]:
excluded_list = top_performers.loc[~rslt_df['NAME'].str.casefold().isin(exclusions)]  # ~ means NOT IN. DROP spex carded athletes

In [449]:
#spexed_list=top_performers

In [450]:
excluded_list.sort_values(['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'], ascending=[True, True, False], inplace=True)
excluded_list['overall_rank'] = 1
excluded_list['overall_rank'] = excluded_list.groupby(['MAPPED_EVENT', 'GENDER'])['overall_rank'].cumsum()

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_82118/1842046915.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excluded_list.sort_values(['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'], ascending=[True, True, False], inplace=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_82118/1842046915.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excluded_list['overall_rank'] = 1
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_82118/1842046915.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [451]:
#spexed_list=spexed_list[(((spexed_list['EVENT']=='400m')&(spexed_list['overall_rank']<7)))|(((spexed_list['EVENT']=='100m')&(spexed_list['overall_rank']<7)))]

In [452]:
#Apply OCTC selection rule: max 6 for 100m/400m and max 3 for all other events

excluded_list = excluded_list[(((excluded_list['MAPPED_EVENT']=='400m')|(excluded_list['MAPPED_EVENT']=='100m'))&(excluded_list['overall_rank']<7))|(~((excluded_list['MAPPED_EVENT']=='400m')|(excluded_list['MAPPED_EVENT']=='100m'))&(excluded_list['overall_rank']<4))]


In [453]:
excluded_list

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5,PERF_SCALAR,overall_rank
8135,11374,"Tan, Bernice",2:16.97,23.0,3,Women 10000 Meter Run Open,2000-11-16,None,Long,Female,...,2131.03,2173.6506,2205.61605,2237.5815,136.97,2036.6806,2068.64605,2100.6115,98.572592,1
8137,11380,"NicAmhlaoibh, Maire",2:16.97,35.0,2,Women 10000 Meter Run Open,1988-02-14,None,Long,Female,...,2131.03,2173.6506,2205.61605,2237.5815,136.97,2036.6806,2068.64605,2100.6115,98.572592,2
8134,11372,"Ng, Yew Cheo",2:16.97,37.0,4,Women 10000 Meter Run Open,1986-11-30,None,Long,Female,...,2131.03,2173.6506,2205.61605,2237.5815,136.97,2036.6806,2068.64605,2100.6115,98.572592,3
9739,13637,"PHAGAMI, KHUM BAHADUR",20.34m,39.0,2,Men 10000 Meter Run Men Senior A,1984-08-09,None,Long,Male,...,1885.55,1923.2610,1951.54425,1979.8275,144.29,1778.9710,1807.25425,1835.5375,97.347591,1
9734,13632,"Mohd, Jamian",35.31m,38.0,6,Men 10000 Meter Run Men Senior A,1985-04-29,None,Long,Male,...,1885.55,1923.2610,1951.54425,1979.8275,144.29,1778.9710,1807.25425,1835.5375,97.347591,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3249,3544,SOLANA LEANN REINKIRSTEN CANDA,00:22.7,0.0,9.0,Triple Jump,None,None,Jump,Female,...,13.46,13.1908,12.98890,12.7870,1150.60,1137.4092,1137.61110,1137.8130,8453.291233,2
3260,3555,SHERYL TOH PEIXUAN,00:23.4,0.0,6.0,Triple Jump,None,None,Jump,Female,...,13.46,13.1908,12.98890,12.7870,1150.60,1137.4092,1137.61110,1137.8130,8453.291233,3
3257,3552,RAO TIANYU,00:28.7,0.0,12.0,Triple Jump,None,None,Jump,Male,...,15.70,15.3860,15.15050,14.9150,1150.60,1135.2140,1135.44950,1135.6850,7233.662420,1
3246,3541,NG JUN JIE,00:15.7,0.0,11.0,Triple Jump,None,None,Jump,Male,...,15.70,15.3860,15.15050,14.9150,1150.60,1135.2140,1135.44950,1135.6850,7233.662420,2


In [454]:
excluded_list.to_csv('octc_all_events.csv', sep=',', encoding='utf-8-sig', index=False)


In [ ]:
# 1. If top athlete>30 (and the only one >30), there are already 6 in list, and next one is already <30 then it means do nothing? 
# 2. If top athlete>30 and is the sole pick.  Next athlete is far beyond 5% band.  Do we add?
# 3. If top athlete>30 and there are less than 6 within 5% band of SEAG benchmark. Do we add that one more who is beyond 5%?
# 4. At least one per gender.  The only pick is far beyond 5%.  Do we add?

# Marinda's request for 6 and 8 place finish analysis

In [4062]:
import pandas_gbq
from google.oauth2 import service_account


credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
)

sql="""
SELECT NAME, RESULT, RANK, EVENT, CATEGORY_EVENT, GENDER, COMPETITION, STAGE
FROM `saa-analytics.results.saa_full`
WHERE STAGE='Final' AND COMPETITION='SEA Games' AND (RANK='3' OR RANK='6' OR RANK='8')
"""

SEAG = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)



Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [4064]:
SEAG.tail(60)

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE
53,Farell Glen Felix Jerus,2.15 m,3,High jump,Jump,Male,SEA Games,Final
54,Sun Soklim,NM,6,High jump,Jump,Female,SEA Games,Final
55,Kobsit Sittichai,2.07 m,6,High jump,Jump,Male,SEA Games,Final
56,Suwandi Wijaya,7.47,6,Long jump,Jump,Male,SEA Games,Final
57,Pok Pisey,4.76,8,Long jump,Jump,Female,SEA Games,Final
58,Sapwaturrahman Sapwaturrahman,7.62,3,Long jump,Jump,Male,SEA Games,Final
59,Bùi Thị Loan,6.02,3,Long jump,Jump,Female,SEA Games,Final
60,Nurul Ashikin Abas,5.67,6,Long jump,Jump,Female,SEA Games,Final
61,John Marvin Aragon Rafols,7.19,8,Long jump,Jump,Male,SEA Games,Final
62,Sunisa Khotseemueang,None,3,Heptathlon,Heptathlon,Female,SEA Games,Final


In [247]:
for i in range(len(SEAG)):
        
    rowIndex = SEAG.index[i]

    input_string=SEAG.iloc[rowIndex,3]  # event is column index 3
    
    metric=SEAG.iloc[rowIndex,1] # result is column index 1
    
    if (metric==None or metric=='NH' or metric=='NM'):  # skip those non-numeric values
        continue
        
   # elif metric=='NH':
    #    continue
        
    print(i, input_string, metric)
        
    out = convert_time(i, input_string, metric)
     
    SEAG.loc[rowIndex, 'Metric'] = out

0 1500m     03:59.40
1 1500m     04:04.3
2 1500m     04:05.4
3 Decathlon 6891
4 100m 10.78
5 100m 11.75
6 100m 11.96
7 100m 10.443
8 200m 24.09
9 200m 23.6
11 200m 21.02
12 200m 21.58
13 400m 46.63
14 400m 48.26
15 400m 57.54
16 400m 1:01.20
17 400m 53.84
18 800m 1:57.98
19 800m 1:53.86
20 800m 2:15.77
21 800m 2:18.25
22 800m 2:09.15
23 800m 1:55.39
24 1500m 4:26.33
25 1500m 4:38.64
26 1500m 5:00.090
27 5000m 17:13.63
28 5000m 18:41.76
29 5000m 14:43.45
30 5000m 15:01.79
31 5000m 18:02.52
32 5000m 15:09.65
33 10000m 33:00.23
34 10000m 32:36.32
35 10000m 38:11.04
36 10000m 37:29.58
37 10000m 35:31.03
38 10000m 31:25.55
39 Marathon 2:35:49
40 Marathon 3:17:36
41 Marathon 3:46:44
42 Marathon 2:52:41
43 Marathon 2:50:27
44 Marathon 2:41:36
45 Shot put 11.69
46 Shot put 12.12
47 Shot put 14.44
48 Shot put 14.65
49 Shot put 17.3
50 Shot put 16.06
52 High jump 2.07 m
53 High jump 2.15 m
54 High jump 1.73
55 High jump 1.95 m
56 Long jump 7.47
57 Long jump 5.67
58 Long jump 6.02
59 Long jump 4.

In [248]:
SEAG

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE,Metric
0,Wahyudi Putra,03:59.40,3,1500m,Mid,Male,SEA Games,Final,239.40
1,Edwin GIRON,04:04.3,6,1500m,Mid,Male,SEA Games,Final,244.30
2,Van Dung GIANG,04:05.4,8,1500m,Mid,Male,SEA Games,Final,245.40
3,Aries TOLEDO,6891,3,Decathlon,Decathlon,Male,SEA Games,Final,6891.00
4,Joshua Hanwei Chua,10.78,6,100m,Sprint,Male,SEA Games,Final,10.78
...,...,...,...,...,...,...,...,...,...
107,"Sukanya Janchaona, Benny Nontanam, Sasipim S...",3:39.29,3,4 x 400m relay,Relay,Female,SEA Games,Final,219.29
108,Nguyễn Thị Huong,11:00.85,3,3000m steeplechase,Steeple,Female,SEA Games,Final,660.85
109,Pandu Sukarya,8:55.05,3,3000m steeplechase,Steeple,Male,SEA Games,Final,535.05
110,Ri Udom,10:36.06,8,3000m steeplechase,Steeple,Male,SEA Games,Final,636.06


In [249]:
SEAG.to_csv('check_variation.csv', encoding='utf-8')

In [250]:
comps = SEAG[SEAG['RANK']=='3']

In [251]:
comps

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE,Metric
0,Wahyudi Putra,03:59.40,3,1500m,Mid,Male,SEA Games,Final,239.400
3,Aries TOLEDO,6891,3,Decathlon,Decathlon,Male,SEA Games,Final,6891.000
5,Trần Thị Nhi Yến,11.75,3,100m,Sprint,Female,SEA Games,Final,11.750
7,Muhammad Haiqal Hanafi,10.443,3,100m,Sprint,Male,SEA Games,Final,10.443
9,Zaidatul Husniah Zulkifli,23.6,3,200m,Sprint,Female,SEA Games,Final,23.600
11,Lalu Muhammad Zohri,21.02,3,200m,Sprint,Male,SEA Games,Final,21.020
13,Frederick Ramirez,46.63,3,400m,Sprint,Male,SEA Games,Final,46.630
17,Nguyễn Thị Hằng,53.84,3,400m,Sprint,Female,SEA Games,Final,53.840
19,Wan Muhammad Fazri Wan Zahari,1:53.86,3,800m,Mid,Male,SEA Games,Final,113.860
22,Goh Chui Ling,2:09.15,3,800m,Mid,Female,SEA Games,Final,129.150


In [252]:
new_comps = comps[['EVENT', 'GENDER', 'Metric']]

In [253]:
#comps=SEAG.groupby(['EVENT', 'GENDER'])['Metric'].min()


In [254]:
df2 = SEAG.merge(new_comps, on=['EVENT', 'GENDER'], how='left')


In [255]:
df2

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE,Metric_x,Metric_y
0,Wahyudi Putra,03:59.40,3,1500m,Mid,Male,SEA Games,Final,239.40,239.400
1,Edwin GIRON,04:04.3,6,1500m,Mid,Male,SEA Games,Final,244.30,239.400
2,Van Dung GIANG,04:05.4,8,1500m,Mid,Male,SEA Games,Final,245.40,239.400
3,Aries TOLEDO,6891,3,Decathlon,Decathlon,Male,SEA Games,Final,6891.00,6891.000
4,Joshua Hanwei Chua,10.78,6,100m,Sprint,Male,SEA Games,Final,10.78,10.443
...,...,...,...,...,...,...,...,...,...,...
107,"Sukanya Janchaona, Benny Nontanam, Sasipim S...",3:39.29,3,4 x 400m relay,Relay,Female,SEA Games,Final,219.29,219.290
108,Nguyễn Thị Huong,11:00.85,3,3000m steeplechase,Steeple,Female,SEA Games,Final,660.85,660.850
109,Pandu Sukarya,8:55.05,3,3000m steeplechase,Steeple,Male,SEA Games,Final,535.05,535.050
110,Ri Udom,10:36.06,8,3000m steeplechase,Steeple,Male,SEA Games,Final,636.06,535.050


In [256]:
df2.to_csv('check_metric.csv', encoding='utf-8')

In [257]:
df2['% VARIATION']=(df2['Metric_x']/df2['Metric_y']*100-100)

In [258]:
df2

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE,Metric_x,Metric_y,% VARIATION
0,Wahyudi Putra,03:59.40,3,1500m,Mid,Male,SEA Games,Final,239.40,239.400,0.000000
1,Edwin GIRON,04:04.3,6,1500m,Mid,Male,SEA Games,Final,244.30,239.400,2.046784
2,Van Dung GIANG,04:05.4,8,1500m,Mid,Male,SEA Games,Final,245.40,239.400,2.506266
3,Aries TOLEDO,6891,3,Decathlon,Decathlon,Male,SEA Games,Final,6891.00,6891.000,0.000000
4,Joshua Hanwei Chua,10.78,6,100m,Sprint,Male,SEA Games,Final,10.78,10.443,3.227042
...,...,...,...,...,...,...,...,...,...,...,...
107,"Sukanya Janchaona, Benny Nontanam, Sasipim S...",3:39.29,3,4 x 400m relay,Relay,Female,SEA Games,Final,219.29,219.290,0.000000
108,Nguyễn Thị Huong,11:00.85,3,3000m steeplechase,Steeple,Female,SEA Games,Final,660.85,660.850,0.000000
109,Pandu Sukarya,8:55.05,3,3000m steeplechase,Steeple,Male,SEA Games,Final,535.05,535.050,0.000000
110,Ri Udom,10:36.06,8,3000m steeplechase,Steeple,Male,SEA Games,Final,636.06,535.050,18.878609


In [259]:
df2.to_csv('SEAG_variation.csv', sep=',', encoding='utf-8-sig', index=False)


# Convert timing format from seconds to HMSf

In [4803]:
datetime.datetime.fromtimestamp(120).strftime('%M:%S.%f')
        

'32:00.000000'

In [4808]:
def convert(seconds):
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
     
    return "%d:%02d:%02d" % (hour, minutes, seconds)

In [5295]:
import datetime
def format_result(result):
    date = datetime.datetime.utcfromtimestamp(result)
    output = datetime.datetime.strftime(date, "%H:%M:%S.%f")
    return output
print(format_result(113.86))

00:01:53.860000


# Clean manual load data

In [1521]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/Singapore Athletes Events/Additions/')


df_SHA = pd.read_csv("SHA_data2.csv", encoding='latin-1')

In [1522]:
df_SHA

,RANK,TAG_ID,NAME,TEAM,SEED,RESULT,QUALIFICATION,HEAT,LANE,WIND,...,DOB,GROUP,CATEGORY_EVENT,ATHLETE_ID,SOURCE,REMARKS,TIMESTAMP,FREE_FIELD,FREE_FIELD2,FREE_FIELD3
0,NaN,NaN,"YEE, MELISSA",Singapore,NaN,11.44m,NaN,NaN,NaN,NaN,...,NaN,NaN,Throw,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,CHEW YIU TSE JADE,Singapore,NaN,1.65m,NaN,NaN,NaN,NaN,...,NaN,NaN,Jump,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1523]:
df_SHA['NAME'] = df_SHA['NAME'].str.replace('\xa0', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.replace('\r', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.replace('\n', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.strip()


In [1524]:
df_SHA.to_csv('SHA_data2.csv', index=False, encoding='utf-8')

# Read a csv of name list variations

In [2566]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')

names = pd.read_csv("name_variations.csv")

In [2567]:
names

,VARIATION,NAME
0,^Harry\sIrfan\sCurra$,HARRY IRFAN CURRAN
1,"Curran, Harry Irfann",HARRY IRFAN CURRAN
2,"^Curran,\sHarry$",HARRY IRFAN CURRAN
3,Harry Irfan Curran Q,HARRY IRFAN CURRAN
4,Wei Jun Huang,HUANG WEI JUN
...,...,...
182,Chua Hsin-Wen Clara,"Chua Hsin-Wen, Clara"
183,Chua Hsin-Wen Clara Q,"Chua Hsin-Wen, Clara"
184,"HSIN-WEN CLARA, Chua","Chua Hsin-Wen, Clara"
185,CHUA HSIN-WEN CLARA,"Chua Hsin-Wen, Clara"


In [ ]:
lines = [line.strip() for line in open('file.csv')]
for x in lines:
    match=re.search(r'^_.*_$',x)
    if match: print x

In [ ]:
df['NAME'] = df['NAME'].replace(regex=r'^Harry\sIrfan\sCurra$', value='HARRY IRFAN CURRAN')


In [2568]:
for index, row in names.iterrows():
    print(row['VARIATION'])
    

^Harry\sIrfan\sCurra$
Curran, Harry Irfann
^Curran,\sHarry$
Harry Irfan Curran Q
Wei Jun Huang
HUANG, WEI JUN
., Hariharan
S/O KrishnS/O Krishna, Hariharan
KRISHNAN, HARIHARAN S/O
Andrew George Medina
Medina, Andrew
Medina, Andrew George
George Medina, Andrew
George George Medina, Andrew
Gabriel Lee
Lee, Gabriel
Jun Yu Low
Low Jun Yu
Low, Jun Yu
Caleb Hia
Melvin, Wong
MELVIN, WONG
Conrad Kangli Emery
Conrad, Kangli Emery
Kangli Emery Conrad
Conrad Kangli Emery Q
Mun Jern Wei
Mun, Jern Wei Ivan
Mun, Ivan
Wong Yijie Lucas
Wong, Yijie Lucas
Tung Hon Andrew Pak
Pak, Andrew
Brandon Heng Fu Hong
Heng, Brandon
Brandon Heng Fu Hong
HENG, BRANDON
HENG, FU HONG, BRANDON
HENG, FU HONG BRANDON
Mohamed, Haja Fayiz
Osman, Amir Rusyaidi
nan
Goh, Shaun
Shaun Goh
Ethan Yan
TAN, AARON JUSTIN
Tan, Aaron Justin
Rui Yong Soh
YEE, ERIC CHUN WAI
Yee, Eric
YEE CHUN WAI, ERIC
Eric Yee Chun Wai
Yee Chun Wai, Eric
Yee, Chun Wai, Eric
Yee Chun Wai, Eric
YEE, ERIC CHUN WAI
Lim, Oliver
Jie Cong Jayden, Tan
TAN , JI